# Reinforcement learning
Reinforcement learning is a framework in which an agent learns to make decisions by interacting with an environment in order to maximize cumulative reward. The agent observes the current state, selects an action, receives a reward, and transitions to a new state. Over time, it learns a policy that maximizes long-term returns through trial and error.

## Terminology
**Agent**: the entity that makes decisions, performs actions in the environment, receives rewards, and learns from the consequences of its actions

**Environment**: everything external to the agent that it can interact with

**State ($s$)**: all the information needed to describe the current situation of the environment. The agent will take action based on the current state, and an agent action (most of the time) will cause the transition to a new state. A state should be Markovian

**Action ($a$)**: the decision taken by the agent after assessing the current state that can affect the environment. An action can be either discrete or continuous

**Reward ($r$)**: a feedback given to an agent after it performs an action. The reward can be positive or negative based on the action taken and the resulting state, which tells the agent how good or bad its action was. A reward can be immediate or delayed. The goal of the agent is to maximize the cumulative reward. The reward signal is the way of communicating to the agent what we want achieved, not how we want it achieved.

**Discount factor ($\gamma$)**: a number between 0 and 1 that balances the future reward value based on the number of actions required. The more actions required, the smaller the rewards will be. Larger $\gamma$ means the algorithm is patient and will look for long term reward, where smaller $\gamma$ means the algorithm is impatient and will look for short term reward. The discount factor ensures the reward convergence in infinite horizon problems 

**Return ($G_t$)**: the total accumulated reward with discount after a timestep $t$, where
$$G_t= r_{t+1} + \gamma r_{t+2} + \gamma^2 r_{t+3} + ... + \gamma^{n-1} r_{t+n},$$
where $r_{t+1}$ represents the reward after $t$th action is taken followed by $n$ total actions afterward

**Policy ($\pi(s)$)**: a function that takes in the current state of the agent, $s$, and return an action, $a$ to perform. The agent makes decision based on a policy. The goal of RL is to find an optimal policy $\pi^*$ that maximizes expected return from every state

**Trajectory**: the sequence of states, actions, and rewards the agent experiences

**Episode**: a trajectory that ends in a terminal state

**Exploration**: the agent tries new or less-known actions to discover the environment

**Exploitation**: the agent select the best known action to maximize the reward to its current knowledge

## General RL workflow
For a general RL workflow, the agent observes the current state of the environment and selects an action based on a policy at each time step. The environment then responds by transitioning to a new state and providing a reward that reflects the quality of the action taken. The agent uses this reward, along with the new state, to update its understanding of the environment and improve its policy

This cycle of observing, acting, receiving rewards, and learning continues over many episodes (trial and error), allowing the agent to gradually learn a policy that maximizes long-term cumulative reward. Key components of this process include exploration (trying new actions to gather information) and exploitation (choosing the best-known action to maximize reward). Over time, the agent aims to find a balance between the two and converge toward an optimal behaviour

<img src="https://www.scribbr.com/wp-content/uploads/2023/08/the-general-framework-of-reinforcement-learning.webp" width=500>

# Multi-armed bandit
Multi-armed bandit is the simplest reinforcement learning problem. An agent will choose between $k$ different actions at each timestep, and recieves a reward based on the action chosen, but the reward distributions are unknown and different for each action. The goal of the agent is to maximize the cumulative rewards in the given amount of steps

<img src="https://miro.medium.com/v2/resize:fit:894/1*ZS_craAiKCJzFj9dQ9RaYQ.png" width=500>

## Action-Value

The value of an action, $q(a)$, is the expected reward received when action $a$ is taken:

$$q(a) = \mathbb{E}[R \mid A = a]$$

* $q(a)$: the expected reward of taking action $a$
* In multi-armed bandit problems, there is no concept of state or policy, so the value depends only on the action itself.

### Sample-Average Method
In reinforcement learning, the true action values $q(a)$ are unknown and must be estimated through repeated interactions with the environment.  
One simple estimation approach is the sample-average method, where the estimated value $\hat{q}_t(a)$ at time $t$ is:

$$\hat{q}_t(a) = \frac{\sum_{i=1}^{t-1} \mathbb{1}[A_i = a] \cdot r_i}{\sum_{i=1}^{t-1} \mathbb{1}[A_i = a]} = \frac{\text{Cumulative reward recieved from action $a$ before timestep $t$}}{\text{Total number of time the action $a$ is taken before timestep $t$}}$$

* $A_i$: the action taken at time step $i$
* $r_i$: the reward received at time step $i$
* $\mathbb{1}[A_i = a]$: an **indicator function** that equals 1 if action $a$ was taken at time $i$, and 0 otherwise

As the number of samples increases, $\hat{q}_t(a)$ converges to the true expected value $q(a)$, by the law of large numbers.

#### Incremental Updates

When learning to estimate the expected reward of each action, we need to update our estimate each time a new reward is observed. One issue with the sample-average method is that it requires storing all past rewards for each action to compute $\hat{q}_t(a)$. As the number of trials increases, the required memory grows linearly, which becomes inefficient. To solve this, we can rewrite the sample-average update in a recursive (incremental) form, where

$$\hat{q}_{t+1} = \hat{q}_t + \alpha_t \left(r_t - \hat{q}_t\right)$$

* $\hat{q}_{t+1}$: the updated estimate of the action-value after time step $t$
* $\hat{q}_t$: the previous estimate before seeing the latest reward
* $r_t$: the reward received at time step $t$
* $\alpha_t$: the step size or learning rate, a value between 0 and 1 that determines how much the estimate is updated

This form only requires storing the current estimate and does not require keeping track of all past rewards, making it computationally efficient.

## Non-Stationary Problem
A problem is said to be stationary if the reward distribution for each action remains the same over time. However, in many real world RL problems, the environment is non-stationary, meaning the reward distributions change over time. In such cases, an agent must be able to adapt its action-value estimates to reflect the most recent outcomes more than older ones. The incremental update rule can be written in a recursive form

$$\hat{q}_{t+1} = (1-\alpha)^t\hat{q}_1 + \sum^{t}_{i=1}\alpha (1-\alpha)^{t-i} r_i$$

* $\alpha$: a constant step size between 0 and 1
* $r_i$: the reward recieved after taking the action at timestep $i$

This formula represents the exponentially decaying weighted average of past rewards, where recent rewards are given more significance, and older rewards gradually “fade out” due to the $(1 - \alpha)^{t-i}$ decay factor.

## Exploration and Exploitation Tradeoff
One key challenge in reinforcement learning is deciding when to explore and when to exploit, as an agent cannot do both simultaneously. Exploration helps the agent improve its knowledge about the environment, which can lead to greater rewards in the long term. Exploitation, on the other hand, involves leveraging the agent’s current knowledge to maximize immediate rewards. An optimal policy should strike a balance between exploration and exploitation based on the agent’s current knowledge and state, in order to maximize cumulative reward over time.


### Greedy
The greedy policy is simple as the agent always exploits by choosing the action with the highest estimated reward, without any exploration. While this strategy can work in very simple or well-understood environments, it often performs poorly in more complex or uncertain problems, because the agent never tries new actions and thus fails to discover potentially better options or adapt when conditions change.

### $\epsilon$ Greedy
The $\epsilon$-greedy policy is a variation of the greedy policy that introduces a small amount of exploration.  
At each time step, the agent exploits with probability $1 - \epsilon$, and explores by choosing a random action) with probability $\epsilon$. This policy helps the agent avoid getting stuck with suboptimal actions by occasionally trying alternatives.This can be written as

$$
A_t =
\begin{cases}
\arg\max_a \hat{q}_t(a) & \text{with probability } 1 - \varepsilon \\
\text{a random action} & \text{with probability } \varepsilon
\end{cases}
$$

In general, the $\epsilon$ greedy policy will perform better than greedy in the long run as it gains more knowledge about the environment through exploration

### Optimistic Initial Value
Optimistic initial value is another strategy that balances exploration and exploitation by encouraging the agent to explore early. The idea is to initialize the estimated value of all actions to a number higher than the actual maximum reward. This causes the agent to optimistically assume all actions are promising, so it will try each action early on to verify its assumption. In the early timesteps, the agent explores all actions because it believes "every action might be great." Over time, as the agent gathers more data and updates its estimates, the action values converge to their true values, and the agent naturally shifts to exploitation.

However, here are some issues of optimistic initial value:
1. Only encourages early exploration: Once the agent’s estimates stabilize, it behaves greedily and stops exploring, which can lead to suboptimal policies if early exploration missed better actions
2. Poor performance in non-stationary environments: In environments where the reward distributions change over time, the agent may stop exploring too early and fail to adapt
3. Choosing the initial optimistic value is tricky: It must be high enough to encourage exploration, but not too high to delay convergence. Often, the true maximum reward is unknown, making this hard to tune

### Upper-Confidence Bound Action Selection (UCB)

UCB is a method for balancing exploration and exploitation by considering both the current value estimates and the uncertainty in those estimates. At each time step, the agent selects the action using the following rule:

$$
A_t = \arg\max_a \left[ \hat{q}_t(a) + c \sqrt{\frac{\ln t}{N_t(a)}} \right]
$$

* $A_t$: the action selected at time step $t$
* $\hat{q}_t(a)$: the current estimated value of action $a$
* $c$: a user-defined parameter that controls the degree of exploration. Larger $c$ encourages exploration; smaller $c$ favors exploitation
* $t$: the current time step
* $N_t(a)$: the number of times action $a$ has been selected so far

In this formula, $\hat{q}_t(a)$ is the exploitation term, which indicates the agent's current estimate of how good action $a$ is. The term $c \sqrt{\frac{\ln t}{N_t(a)}}$ is the exploration bonus, which is large when action $a$ has been selected only a few times (low $N_t(a)$), encouraging the agent to explore it.

The logarithmic (unbounded) growth in $\ln t$ ensures that all actions will eventually be explored, but actions with lower estimated value or that have already been selected many times will be chosen less frequently over time. This way, UCB systematically prioritizes actions with high upper confidence bounds

Note: UCB is an deterministic algorithm

<img src="https://media.geeksforgeeks.org/wp-content/uploads/20200126023259/Screenshot-2020-01-26-at-2.32.38-AM.png" width=500>

# Markov Decision Process (MDP)
In the bandit problem, the agent selects actions in the same static environment, where each action yields a reward independent of past actions or time. However, in many real-world problems, the environment is dynamic, and the agent must select different actions depending on the current situation. Markov Decision Processes (MDPs) provide a classical formalization for sequential decision making tasks, where each action affects not only the immediate reward but also the next state and all future actions and rewards as well.

All MDPs are Markovian, meaning the future state and reward depend only on the current state and action, not on the full history. Therefore, knowing the current state is sufficient for optimal decision making, and remembering earlier states does not improve predictions about the future.

## The Agent-Environment Interface
In MDPs, the agent and environment interact continuously in either discrete or continuous time. At each timestep $t$, the agent selects an action $A_t$ based on the current state of the environment $S_t$. As the result of its action, the agent will recieve a reward $r_t$ from the envirnment at the next timestep $t+1$, and the environment will transition into a new state $S_{t+1}$. Then, the agent selection its next action $A_{t+1}$ based on the new states. The sequence of states, actions, rewards is called a trajectory

<img src="https://www.researchgate.net/publication/340694475/figure/fig2/AS:938161513455616@1600686543282/The-agent-environment-interaction-in-reinforcement-learning.jpg" width=500>

## Dynamics of MDPs
In finite MDPs, where the sets of states, actions, and rewards are all finite, the dynamics of the environment are defined by the transition probability function:

$$p(s', r \mid s, a)$$

This represents the joint probability of transitioning to state $s'$ and receiving reward $r$, given that the agent is in state $s$ and takes action $a$. In other words, it defines:
1. the likelihood of moving to a specific next state $s'$
2. and receiving a particular reward $r$
based on the current state–action pair $(s, a)$.

This formulation captures the Markov property, where the next state and reward depend only on the current state and action, not on any earlier history.

## Episodic and Continuing Tasks
There are two types of tasks in RL, episodic tasks and continuing tasks.

The tasks are episodic when the agent–environment interaction breaks naturally into subsequences, where each episodes begins from a standard starting state or a sample from a standard distribution of starting states and will eventual reach a terminal state. Each episode begins independently of how the previous one ended.

The tasks are continuing when the agent–environment interaction cannot be broken down into subsequences and will go on without an ending
## Episodic and Continuing Tasks

In reinforcement learning, tasks are generally classified into two types: episodic and continuing.

A task is episodic when the agent–environment interaction is naturally divided into distinct episodes. Each episode begins in a standard starting state (or sampled from a starting state distribution), proceeds through a sequence of interactions, and eventually reaches a terminal state. After an episode ends, the environment resets, and the next episode begins independently of how the previous one ended.

A task is continuing when there is no natural endpoint, where the agent–environment interaction continues indefinitely without reaching a terminal state. This setting is common in real-world systems that operate continuously, such as online recommendation engines or stock trading agents.

<img src="https://av.tib.eu/thumbnail/63100" width=500>

## Goal of Reinforcement Learning

The goal of reinforcement learning at any time step $t$ is to maximize the expected return, denoted $G_t$, which represents the cumulative reward the agent can expect to receive starting from that time step.

In episodic tasks, each episode has a fixed or variable length and terminates at some final time step $T$.  
Since the episode ends, the return is always finite:

$$G_t = r_{t+1} + r_{t+2} + \dots + r_T$$


In continuing tasks, the agent–environment interaction does not end, so we introduce a discount factor $\gamma \in [0, 1)$ to ensure the return remains finite, where

$$G_t = r_{t+1} + \gamma r_{t+2} + \gamma^2 r_{t+3} + \dots = \sum_{k=0}^{\infty} \gamma^k r_{t+k+1}$$

* A larger discount factor ($\gamma \to 1$) makes the agent more far-sighted, valuing long-term rewards.
* A smaller discount factor makes the agent more short-sighted, prioritizing immediate rewards.

Note: $\frac{R_{\text{max}}}{1 - \gamma}$ is an upper bound on return only if each reward is bounded by $R_{\text{max}}$, i.e., $r_t \leq R_{\text{max}}$.


### Recursive Form of Return
The return can also be defined recursively, where

$$G_t = r_{t+1} + \gamma G_{t+1}$$

This recursive relationship is fundamental in deriving value functions and forms the basis of many RL algorithms.

# Policies and Value Functions
## Policies
A policy, denoted by $\pi$, defines the agent’s behaviour by specifying a mapping from states to a probability distribution over actions. Formally, given the current state $s$, the policy defines

$$\pi(a \mid s)$$

This expression represents the probability of selecting action $a$ when the agent is in state $s$.

Policies can be:
* Deterministic: where $\pi(s)$ directly maps to a specific action.
* Stochastic: where $\pi(a \mid s)$ gives a probability distribution over actions.

Note: A policy is typically a function of only the current state $s$, due to the Markov property. If the policy depends on more than the current state (e.g., past states), then the problem setting is no longer a MDP unless those inputs are encoded into the current state.

## State Value Function
The state value function, denoted by $v_\pi(s)$, represents the expected return when the agent starts in state $s$ at time step $t$ and follows a policy $\pi$ thereafter. It is defined as

$$v_\pi(s) = \mathbb{E}_\pi \left[ G_t \mid s_t = s \right] = \mathbb{E}_\pi \left[ \sum_{k=0}^\infty \gamma^k r_{t+k+1} \mid s_t = s \right]$$


## Action Value Function
The action value function, denoted by $Q_\pi(s, a)$, represents the expected return when the agent starts in state $s$, takes action $a$, and then follows policy $\pi$ thereafter. It is defined as

$$Q_\pi(s, a) = \mathbb{E}_\pi \left[ G_t \mid s_t = s, a_t = a \right] = \mathbb{E}_\pi \left[ \sum_{k=0}^\infty \gamma^k r_{t+k+1} \mid s_t = s, a_t = a \right]$$

## Bellman Equation
The Bellman equation provides a recursive formulation of the state value function and action value function. Instead of computing returns as infinite sums of future rewards, the Bellman equations allow us to compute value functions based on expected immediate reward plus the discounted value of the next states.

### Bellman Equation for the State Value Function
The Bellman equation for the state value function expresses the value of a state $s$ under a policy $\pi$ in terms of
1. All the actions available in $s$
2. The probability of choosing each action under policy $\pi$,
3. The environment's transition dynamics/probability $p(s', r \mid s, a)$,
4. The values of successor states $v_\pi(s')$.

It is defined as
$$v_\pi(s) = \mathbb{E}_\pi \left[ G_t \mid s_t = s \right] = \sum_{a} \pi(a \mid s) \sum_{s'} \sum_{r} p(s', r \mid s, a) \left[ r + \gamma v_\pi(s') \right]$$

* $s$, $s'$: the current and next state respectively
* $v_\pi(s)$: expected return starting from state $s$ and following policy $\pi$
* $\pi(a \mid s)$: probability of taking action $a$ in state $s$
* $p(s', r \mid s, a)$: probability of transitioning to state $s'$ and receiving reward $r$ given current state $s$ and action $a$
* $\gamma$: discount factor, $0 \leq \gamma < 1$

At a high level, the Bellman equation follows
1. From the current state $s$, consider all possible actions $a$ (weighted by the policy).
2. For each action, consider all possible resulting next states $s'$ and rewards $r$ (weighted by the environment's dynamics).
3. For each possible $(s', r)$ pair, compute the expected return, which is the immediate reward $r$ plus the discounted future value $v_\pi(s')$.
4. Compute the weighted sum all of these to compute $v_\pi(s)$.

Note: $\pi(a \mid s)$ is the probability of taking action $a$ in state $s$ under policy $\pi$ and $p(s', r \mid s, a)$ is the probability of transitioning to next state $s'$ and receiving reward $r$, given current state $s$ and action $a$. In the equation, we are summing over all possible immediate outcomes, which are the actions the agent might take, the rewards it might receive, and the next states it might reach. This allows the equation to "look ahead" one step into the future and compute the expected value of that step, using $\pi(a \mid s)$ to weigh each action and $p(s', r \mid s, a)$ to weigh each possible outcome of that action. This can be visualized with the Backup Diagram.

<img src="https://goodboychan.github.io/images/backup_diagram_for_v.png" width=300>

The result is a weighted average of the immediate reward plus the discounted value of future states. This recursive structure can be thought of as expanding a search tree, where each branch corresponds to a possible action and outcome.

#### Notational Notes
* We omit the time index $t$ in $v_\pi(s)$ because under the Markov property, the value of a state depends only on the current state, not on the specific time step.
* The summation over $r$ is used when the reward distribution is stochastic; in deterministic cases, this may be dropped.

### Bellman Equation for the Action-Value Function
Similarly, the action-value function can also be expressed in a recursive form using the Bellman equation.  
It defines the expected return starting from state $s$, taking a specific action $a$, and then following policy $\pi$ thereafter:

$$q_\pi(s, a) = \mathbb{E}_\pi \left[ G_t \mid s_t = s, a_t = a \right] = \sum_{s'} \sum_{r} p(s', r \mid s, a) \left[ r + \gamma \sum_{a'} \pi(a' \mid s') q_\pi(s', a') \right] = \sum_{s'} \sum_{r} p(s', r \mid s, a) \left[ r + \gamma v_{\pi}(s') \right]$$

* There is no policy term in the outer expectation because the first action $a$ is already fixed (we are evaluating $q_\pi(s, a)$ for a specific action).
* The recursion comes into play after the transition to the next state $s'$, where the agent resumes following policy $\pi$. For each possible next state $s'$ and reward $r$, we
    1. Compute the immediate reward $r$
    2. Add the discounted expected value of the next state, where the value is a weighted sum over all possible next actions $a'$ under policy $\pi$
    
This gives a complete recursive expression for $q_\pi(s, a)$, based on the one-step lookahead and expected future return.


In summary, the Bellman equation leverages the recursive structure of MDPs to transform an infinite sum of future rewards into a system of linear equations. While this makes value computation more tractable in theory, solving this system exactly becomes computationally infeasible in large-scale problems due to the exponential growth in the number of states and actions.

### Relationship Between State-Value and Action Value
1. State value expressed in terms of action value: $v(s)$ is the weighted sum of $q(s,a)$ across all possible actions weighted by the policy
$$v_{\pi}(s) = \sum_{a} \pi(a|s) q_{\pi}(s, a)$$

2. Action value expressed in terms of next state value: $q(s,a)$ is the expected immediate reward plus the discounted value of next state $s'$
$$q_{\pi}(s, a) = \sum_{s', r} p(s', r | s, a) \left[ r + \gamma v_{\pi}(s') \right]$$

## Optimal Policy
The goal of reinforcement learning is to find an optimal policy that maximizes the expected cumulative reward. A policy $\pi$ is said to be better than or equal to another policy $\pi'$ if, for every state $s$, the expected return of following $\pi$ is at least as high as that of following $\pi'$. Formally, $\pi \geq \pi'$ if and only if

$$ v_\pi(s) \geq v_{\pi'}(s) \quad \text{for all } s \in \mathcal{S}$$

An optimal policy, denoted by $\pi_*$, is a policy that is better than or equal to all other policies. That is, for any other policy $\pi'$

$$v_{\pi_*}(s) \geq v_{\pi'}(s) \quad \text{for all } s \in \mathcal{S}$$

There always exists at least one optimal policy, and in some cases, multiple optimal policies may exist, all yielding the same optimal state-value function, $v_*(s)$, and optimal action-value function $q_*(s, a)$.

### Optimal Value Functions
The optimal state-value function, denoted by $v_*(s)$, represents the maximum expected return that can be achieved from state $s$ by among all policies, where

$$v_*(s) = \max_{\pi} v_\pi(s) \quad \text{for all } s \in \mathcal{S}$$

Similarly, the optimal action-value function, denoted by $q_*(s, a)$, represents the maximum expected return achievable from state $s$ by taking action $a$ and then following the best possible policy thereafter:

$$q_*(s, a) = \max_{\pi} q_\pi(s, a) \quad \text{for all } s \in \mathcal{S}, a \in \mathcal{A}$$

### Bellman Optimality Equations

We can express the Bellman equations for the optimal value functions without referencing any specific policy. These are known as the Bellman Optimality Equations.

The optimal state-value function $v_*(s)$ satisfies:

$$v_*(s) = \max_{a} \sum_{s'} \sum_{r} p(s', r \mid s, a) \left[ r + \gamma v_*(s') \right]$$

This equation tells us that under an optimal policy, the value of a state equals the maximum expected return achievable by taking the best possible action from that state. In other words, the best action in any state is the one that leads to the highest expect state value.

The optimal action-value function $q_*(s, a)$ satisfies:

$$q_*(s, a) = \sum_{s'} \sum_{r} p(s', r \mid s, a) \left[ r + \gamma \max_{a'} q_*(s', a') \right]$$

This equation expresses the value of taking action $a$ in state $s$ as the expected immediate reward plus the best possible value achievable from the resulting next state $s'$, by taking the best action $a'$ at that point. Since the current state and action $(s, a)$ are already fixed, there is no choice to be made immediately, the choice comes at the next decision point.

It is essential to express these equations without referencing any policy because the goal of reinforcement learning is to discover the optimal policy. We cannot write the equations in terms of a policy that we don't yet know. The Bellman Optimality Equations define the criteria that an optimal policy must satisfy, and solving them (exactly or approximately) allows us to find the best possible decisions at each state.

### Deriving the Optimal Policy from Optimal Value Functions

Once we have the optimal value functions, it is straightforward to derive the optimal policy from them. The optimal policy $\pi_*$ selects the action in each state that maximizes the expected return, based on the optimal state-value function $v_*(s)$, where

$$\pi_*(s) = \arg\max_a \sum_{s'} \sum_{r} p(s', r \mid s, a) \left[ r + \gamma v_*(s') \right]$$

This means that the optimal policy always chooses the action that leads to the highest expected state value in the next state.

Using the optimal action-value function $q_*(s, a)$, the optimal policy can be written more simply as

$$\pi_*(s) = \arg\max_a q_*(s, a)$$

This means the optimal policy selects the action that has the highest action value for the current state.

In both cases, the agent behaves greedily with respect to the optimal value function, where it chooses the action that promises the greatest long-term reward.

However, in most real-world scenarios, obtaining the optimal policy by directly solving the Bellman equations is unrealistic because it requires

1. Substantial knowledge of the environment, including the transition dynamics $p(s', r \mid s, a)$, which we often do not have access to in practice.
2. Solving the Bellman equations involves extremely high computational and memory demands, especially in environments with large or continuous state and action spaces.

As a result, in most practical applications, we use approximation methods to learn a "good enough" policy or value function, rather than attempting to compute the exact optimal values.

# Policy Evaluation & Improvement 

In reinforcement learning, two fundamental tasks are:
* Policy Evaluation: Determining how good a given policy $\pi$ is by computing its state-value function $v_\pi$.
* Policy Improvement: Improving a policy by iteratively producing strictly better policies until reaching an good enough or optimal policy.

Dynamic Programming (DP) methods can be used to solve both tasks if we have access to the environment's dynamics $p(s', r \mid s, a)$.

## Iterative Policy Evaluation
To evaluate the state values under a policy $\pi$, we use the Bellman equation for the value function, where

$$v_\pi(s) = \sum_{a} \pi(a \mid s) \sum_{s'} \sum_{r} p(s', r \mid s, a) \left[ r + \gamma v_\pi(s') \right]$$

Since computing $v_\pi$ exactly is often impractical, we use an iterative approximation, where

$$v_{k+1}(s) = \sum_{a} \pi(a \mid s) \sum_{s'} \sum_{r} p(s', r \mid s, a) \left[ r + \gamma v_k(s') \right]$$

Each iteration updates the estimated value of every state once, using the latest available estimates. With enough iterations, the values are guaranteed to converge to the true value function $v_\pi$. The key idea of policy evaluation is to use observed (or expected) rewards and the estimated state values one step ahead to iteratively update state values until they converge.

### Algorithm: Iterative Policy Evaluation (Given a Policy $\pi$)

1. Initialize two arrays, $v_k$ and $v_{k+1}$, to store the old and updated state values for all states.
2. Randomly initialize the state values (or initialize all to zero).
3. For each state $s$, compute a new value using
   $$v_{k+1}(s) = \sum_{a} \pi(a \mid s) \sum_{s'} \sum_{r} p(s', r \mid s, a) \left[ r + \gamma v_k(s') \right]$$
4. Store all new values in $v_{k+1}$.
5. After updating all states, replace $v_k$ with $v_{k+1}$ (set $v_k = v_{k+1}$).
6. Repeat steps 3–5 until the values converge, meaning the change between $v_k$ and $v_{k+1}$ is smaller than a chosen threshold.

A unique value function $v_\pi$ is guaranteed to exist for any given policy $\pi$ as long as the task is episodic or the discount factor satisfies $\gamma < 1$. Under these conditions, the iterative updates are guaranteed to converge to the true values.

## Policy Improvement
The primary goal of computing the value function of a policy is to help us find a better policy. Given the state-value function $v_\pi$ of any arbitrary policy $\pi$, we can construct a new policy $\pi'$ by acting greedily with respect to $v_\pi$. That is, in each state, $\pi'$ selects the action that yields the highest expected return based on the current value estimates. By doing this, the new policy $\pi'$ is guaranteed to be at least as good as the original policy $\pi$. If the new policy’s value function $v_{\pi'}$ is strictly better than $v_\pi$, we have improved the policy. If $v_{\pi'} = v_\pi$, this means the current policy is already optimal, and no further improvement is possible.

### Policy Improvement Theorem
Formally, given two policies $\pi$ and $\pi'$, the new policy $\pi'$ is guaranteed to be at least as good as $\pi$ if, for every state $s$:

$$q_\pi(s, \pi'(s)) \geq v_\pi(s)$$

This condition means that in every state $s$, the action chosen by $\pi'$, which can be a same or different action chosen by $\pi$, always yields an expected return greater than or equal to the value of following the original policy $\pi$ from that state onward.

As a result,

$$v_{\pi'}(s) \geq v_\pi(s) \quad \text{for all } s$$

### Greedy Policy Improvement
Applying the policy improvement theorem, given an original policy $\pi$ and its state-value function $v_\pi(s)$, we can obtain a better or equally good policy $\pi'$ by selecting actions greedily with respect to $v_\pi(s)$, where

$$\pi'(s) = \arg\max_a q_\pi(s, a)$$

* If $\pi' \neq \pi$, then $\pi'$ is a strictly better policy (strict improvement).
* If $\pi' = \pi$, then the policy is already greedy with respect to its own value function, which indicates that the current policy is optimal and cannot be improved further.

## Policy Iteration
Policy Iteration is an algorithmic process for finding the optimal policy by repeatedly alternating between policy evaluation and policy improvement. The key idea is that by evaluating the current policy and then improving it based on the evaluation, we can iteratively converge to an optimal or sufficiently good policy. In this process, each policy evaluation gives us more accurate value estimates, and each policy improvement ensures the new policy is strictly better or equally good. The process is guaranteed to converge to the optimal policy in finite MDPs.

### Policy Iteration Algorithm

1. Initialize:  
   * Start with an arbitrary policy $\pi$
   * Initialize the state-value function $v$

2. Policy Evaluation: 
   * Compute $v_\pi$ for the current policy using iterative updates until convergence.

3. Policy Improvement:
   * For each state $s$, update the policy greedily with respect to $v_\pi$:  
   $$\pi_{\text{new}}(s) = \arg\max_a \sum_{s'} \sum_r p(s', r \mid s, a) \left[ r + \gamma v_\pi(s') \right]$$

4. Check for Convergence:
   * If the policy hasn’t changed, the optimal policy has been found.
   * Otherwise, set $\pi = \pi_{\text{new}}$ and return to Step 2.

<img src="https://lcalem.github.io/imgs/sutton/gpi_interaction.png" width=500>

# Generalized Policy Iteration
Generalized Policy Iteration  refers to the framework in reinforcement learning where policy evaluation and policy improvement are interact repeatedly and approximately to gradually refine a policy toward optimality. The key idea is that evaluation and improvement do not need to fully complete before the next begins. As long as policy evaluation moves the value function closer to accurate estimates under the current policy, and policy improvement makes the policy greedier with respect to those estimates, the process will converge over time. If both the value function and the policy eventually stabilize, meaning no further updates occur, then the policy must be optimal, and the value function must be the optimal value function.


## Value Iteration
Value iteration is a form of generalized policy iteration that streamlines the process of finding the optimal policy. It addresses the inefficiency in traditional policy iteration, where the policy evaluation step requires multiple sweeps over all states until the value function converges. Instead of fully evaluating the current policy before improving it, value iteration performs a single update to each state's value in each iteration and greedy policy improvement. This reduces computational cost while still ensuring convergence to the optimal policy.

The core update rule is:
$$v_{k+1}(s) = \max_a \sum_{s'} \sum_r p(s', r \mid s, a) \left[ r + \gamma v_k(s') \right]$$

Each update improves the value estimate of state $s$ by considering the best possible action to take, which is already a greedy selection. Once the value function has converged (changes are below a small threshold), the optimal policy can be derived by acting greedily with respect to the final value function

$$\pi_*(s) = \arg\max_a \sum_{s'} \sum_r p(s', r \mid s, a) \left[ r + \gamma v_*(s') \right]$$

Value iteration has the advantage of
1. More efficient than policy iteration, especially in large state spaces.
2. Combines evaluation and improvement into a single update step.
3. Guaranteed to converge to the optimal value function and policy under standard conditions (finite MDP, $\gamma < 1$ or episodic tasks).

# DP Efficiency & Limitations
Dynamic programming methods, such as policy iteration and value iteration, are foundational tools in reinforcement learning for solving MDPs. These methods are guaranteed to converge to the optimal policy under the right conditions and are considered efficient in a theoretical sense.

DP methods are polynomial time algorithms, meaning their computational complexity grows at a manageable rate with respect to the number of states and actions. This makes them efficient in contrast to brute force approaches that might try all possible policies.

There are two types of DP methods, synchronous and asynchronous DP.

* Synchronous DP: the value of all states is updated simultaneously and systematically in each iteration based on the values from the previous iteration. This is conceptually simple and easy to parallelize, but can be inefficient if many states are rarely visited or irrelevant in a given policy.

* Asynchronous DP: only a subset of states is updated at a time. This allows for more targeted updates focusing on states that are frequently visited or have recently changed. Asynchronous methods can often reach good solutions more quickly in practice. However, to guarantee convergence to the correct value function, all states must still be updated sufficiently often over time. This means asynchronous DP does not necessarily reduce the total number of updates needed for convergence, but it allows for smarter scheduling and prioritization of updates. In practice, this leads to more efficient use of computational resources, especially when combined with techniques like prioritized sweeping.

## Limitations of DP
However, "efficiency" of DP in a theoretical sense does not always translate to practicality in real world problems because of the following reasons

1. Curse of Dimensionality: DP methods become intractable in high dimensional state spaces, where the number of possible states grows exponentially with the number of variables describing the state.

2. Requirement of the Transition Probability: DP requires complete knowledge of the environment, specifically the transition probability function, $p(s', r \mid s, a)$. In real world problems, this dynamic model is usually unknown, making pure DP methods inapplicable without approximation or learning.

Dynamic Programming is a powerful baseline, but its real world applicability is often limited. That’s why modern reinforcement learning often turns to approximate dynamic programming, Monte Carlo methods, and temporal difference learning, which relax these constraints.

# Sampling Based Learning Methods
In many real-world reinforcement learning scenarios, we do not have access to the full model of the environment, such as the transition probabilities or reward distributions. As a result, it becomes infeasible to apply planning methods like dynamic programming, which require full knowledge of the environment's dynamics.

Sampling-based learning methods (model-free methods) address this by learning directly from sampled experiences, the actual sequences of states, actions, and rewards encountered by the agent while interacting with the environment. Instead of computing expectations over all possible next states as in DP, sampling based methods approximate value functions or policies using observed outcomes through experiences.

Common examples of sampling-based methods include
* Monte Carlo methods: Learn from complete episodes by averaging returns.
* Temporal-Difference (TD) learning: Learn from bootstrapped estimates using partial episodes

These methods are more scalable and practical in complex environments because they do not require storing or computing the full transition dynamics. However, they also come with trade-offs, such as increased variance in learning and the need for sufficient exploration to obtain a good enough estimate. In summary, sampling-based learning allows an agent to learn optimal behaviour from trial-and-error experience, without needing a complete model of the environment

## Monte Carlo Method
Monte Carlo methods estimate value functions by averaging returns obtained from multiple sampled episodes. These methods rely solely on sequences of states, actions, and rewards generated through actual interactions with the environment. Unlike DP, Monte Carlo methods do not require knowledge of the environment’s dynamics.

Similar to DP, Monte Carlo methods follow the principle of Generalized Policy Iteration, where they alternate between policy evaluation (estimating how good the current policy is) and policy improvement (using the value estimates to make the policy better), eventually converging to an optimal or good enough policy.

### Monte Carlo for Policy Evaluation
To evaluate a policy using the Monte Carlo approach
1. Given a policy, run simulations based on it to generate many episodes.
2. Record the observed returns of each visited stated in the episode.
3. Update the value estimate towards based on the observed return from that state.

Given that episodes terminate, the return $G_t$ at each time step $t$, where the agent is in state $s_t$ can be computed backward from the end of the episode, where
$$G_t = r_{t+1} + \gamma r_{t+2} + \gamma^2 r_{t+3} + \dots + \gamma^{T-t-1} r_T$$

Then, we update the value function of the state $s_t$ to be
$$v(s_t) \leftarrow v(s_t) + \alpha \left[ G_t - v(s_t) \right]$$
where $[ G_t - v(s_t)]$ refers to the error between the observed return and the current estimates, and $\alpha$ is the update step size or learning rate.

With enough samples, the average return for each state converges to the true expected return under the given policy.

### Monte Carlo vs DP
1. No Need for a Model: Monte Carlo methods do not require access to the environment's transition or reward model. They learn directly from sampled episodes, unlike DP which requires complete knowledge of the dynamics.

2. Independent State Updates: In Monte Carlo, the value of each state is computed independently by averaging returns from that specific state. In contrast, DP relies on bootstrapping, where the value of a state is a weighted sum of the values of successor states. This means
   * Monte Carlo computation scales with the number of episodes (and their length), not the size of the state space (the size of MDP).
   * DP computation scales with the number of states and the transition model complexity.

<img src="https://i0.wp.com/roboticseabass.com/wp-content/uploads/2020/08/rl_intro_banner-1.png?fit=1200,448&ssl=1" width=700>

### Monte Carlo for Action-Value Estimation
In DP, state values are sufficient to determine a policy because the model (transition probabilities and rewards) allows us to look one step ahead and select the action that leads to the best expected outcome. However, in Monte Carlo methods, where no model of the environment is available, state values alone are insufficient for decision making because even though we know the all the state values, we do not know the transition probabilities, and thus, do not know which state the agent will end up in after taking an action. Therefore, we must explicitly estimate action-value functions, $q_\pi(s, a)$, to determine which actions are better under a given policy.

To estimate action values
1. Generate episodes using the current policy.
2. For each state–action pair, we record the return following that pair.
3. The estimated action value $q_\pi(s, a)$ is the average of these observed returns.

This is similar to estimating state values, except we condition on both the state and the action taken, where
$$q(s, a) \leftarrow q(s, a) + \alpha \left[ G_t - q(s, a) \right]$$

### Exploring Starts
However, this methods has an critical issue when the agent follows a deterministic policy, and certain state–action pairs may never be visited. As a result, some action values will never be updated, and the agent cannot learn about better actions it never tries. This violates the principle of sufficient exploration, making it impossible to guarantee convergence to the optimal policy.

One common solution in Monte Carlo control is exploring starts, where each episode begins from a randomly chosen state–action pair and follow the policy for the remainder of the episode. This ensures that all state–action pairs have a non-zero probability of being explored. With enough episodes, this guarantees that all $q_\pi(s, a)$ values are visited and accurately estimated.

### Monte Carlo Method for Policy Iteration
Monte Carlo methods can be used to perform policy iteration. This is achieved through sampling episodes and estimating the action values from experience. The algorithm proceeds as follows

1. Initialization
* Initialize an arbitrary policy $\pi$.
* Initialize the action-value function $q_\pi(s, a)$ arbitrarily (e.g., to zero for all $(s, a)$ pairs).

2. Generate an Episode
* Use exploring starts to ensure sufficient coverage of all state-action pairs.
* Generate an episode by starting from a random state-action pair, then follow the current policy $\pi$.
* Record the sequence of $(s_t, a_t, r_{t+1})$ for the entire episode until termination.


3. Monte Carlo Policy Evaluation
* For each state-action pair $(s, a)$ that appears in the episode:
     - Compute the return $G_t$ backward with discount.
     - Update the action-value estimate $q_\pi(s, a)$ as the sample average of all returns observed for that pair:
    $$q_\pi(s, a) \leftarrow q(s, a) + \alpha \left[ G_t - q(s, a) \right]$$

Note: Only the $(s, a)$ pairs actually visited in the episode are updated; others remain unchanged. In theory, an infinite number of episodes ensures accurate value estimation for all pairs. In practice, however, we use limited episodes, meaning early updates may be noisy, but over many iterations, the estimates improve.

4. Policy Improvement
* For each state $s$ visited in the episode, improve the policy by choosing the action with the highest estimated value to make the policy greedy with respect to the current $q_\pi$, where
  $$\pi_{\text{new}}(s) = \arg\max_a q_\pi(s, a)$$

5. Repeat
* Return to Step 2 and continue until the policy converges, or for a fixed number of iterations.

Note: Using one or few episode per iteration is analogous to stochastic gradient descent, where each update may be noisy, but the overall trend improves the policy over time. The use of exploring starts ensures that all state-action pairs have a non-zero chance of being updated, which is critical for convergence guarantees.

### Monte Carlo for Action-Value Without Exploring Starts
In Monte Carlo control with exploring starts, every state-action pair must have a non-zero probability of being the initial pair in an episode. This guarantees that all $(s, a)$ pairs are eventually visited, which is critical for ensuring convergence to the optimal policy. However, exploring starts are impractical in real world problems, especially those with continuous or infinite state-action spaces, like self-driving cars, where it’s impossible to initialize the agent in every possible situation.

### $\boldsymbol{\varepsilon}$-Soft Policies
To ensure sufficient exploration without exploring starts, we use $\varepsilon$-soft policies, which are stochastic policies that assign non-zero probability to all actions in every state, where

$$\pi(a \mid s) \geq \frac{\varepsilon}{|\mathcal{A}(s)|}$$

This guarantees that every action in every state has a chance of being selected with at least probability of $\frac{\varepsilon}{\text{Number of possible actions}}$. Thus, all state-action pairs are eventually explored with enough episodes. Both $\varepsilon$-greedy policy and uniform random policy are examples of $\varepsilon$-soft policies

Using an $\varepsilon$-soft policy ensures exploration and removes the need for exploring starts. However, there’s a trade-off, where the agent can only converge to the optimal $\varepsilon$-soft policy, not the optimal deterministic policy. Even after learning a near-optimal policy, the agent still occasionally explores due to the non-zero $\varepsilon$.

In practice, the performance of the optimal $\varepsilon$-soft policy is often close to that of the optimal deterministic policy, and perform well enough for the need.

### Monte Carlo Control with $\varepsilon$-Soft Policy (Policy Iteration)
The process is similar to standard Monte Carlo policy iteration, with the key difference being the use of stochastic policies:

1. Initialize:
   - Initialize a random $\varepsilon$-soft policy $\pi$.
   - Initialize action-value estimates $q(s, a)$ arbitrarily.

2. Generate Episode:
   - Generate an episode following the current $\varepsilon$-soft policy.
   - Record the sequence of $(s_t, a_t, r_{t+1})$.

3. Policy Evaluation:
   - For each state-action pair in the episode:
     - Compute return $G_t$ and update $q(s, a)$ as the sample average.

4. Policy Improvement:
   - For each state $s$ visited:
     - Make the policy $\varepsilon$-greedy with respect to the current $q(s, a)$:
       $$
       \pi(a \mid s) =
       \begin{cases}
       1 - \varepsilon + \frac{\varepsilon}{|\mathcal{A}(s)|}, & \text{if } a = \arg\max_a q(s, a) \\
       \frac{\varepsilon}{|\mathcal{A}(s)|}, & \text{otherwise}
       \end{cases}
       $$

5. Repeat steps 2–4 until convergence.

Overall, $\varepsilon$-soft policy solves the problem of insufficient exploration with a large number of possible state-action pairs, and is able to achieve a good enough, sub-optimal stochastic policy.

# On-Policy vs Off-Policy Learning
In reinforcement learning, there are two fundamental paradigms for learning from interaction, on-policy and off-policy learning. The key difference lies in whether the policy being learned is the same as the policy used to generate the data.

## On-Policy Learning
In on-policy learning, the agent learns about and improves the same policy that it uses to make decisions. This means the data used for learning is generated by following the current policy, which typically includes a balance of exploitation and exploration. 

**Pros:**
1. Conceptually simpler.
2. Lower variance in value estimates since learning is based on the actual policy being executed.

**Cons:**
1. Exploration is constrained by the policy itself.
2. May learn slower due to limited exploration.

## Off-Policy Learning
In off-policy learning, the agent learns about a different policy than the one used to generate the data. There are two distinct policies involved:

1. Target policy: the policy the agent is trying to learn or evaluate, denoted by $\pi(a \mid s)$
2. Behaviour policy: the policy used to generate data (select actions), denoted by $b(a \mid s)$

This allows the agent to learn an optimal or deterministic target policy while still exploring using a stochastic behaviour policy.

**Pros:**
1. More general and powerful; can learn from data generated by other agents or past experiences.
2. Enables stronger exploration while still converging to an optimal deterministic policy.

**Cons:**
1. Higher variance in updates, especially when using importance sampling.
2. Requires careful alignment between behaviour and target policies.

For off-policy learning to be valid, the behaviour policy must cover all the actions that the target policy might take. That is:
$$\text{If } \pi(a \mid s) > 0, \text{ then } b(a \mid s) > 0$$

This ensures that the agent gets enough data to estimate the value of the target policy correctly. If the behaviour policy never explores certain actions that the target policy might select, it is impossible to evaluate or improve the target policy accurately.

## Importance Sampling
Importance sampling is a fundamental technique used in off-policy reinforcement learning. It allows us to estimate expectations under one probability distribution (the target policy) using samples drawn from a different distribution (the behaviour policy). This is crucial in off-policy learning, where the agent aims to learn or evaluate a policy that is different from the one used to generate data.

In off-policy learning, we want to compute the expected return or value function under the target policy $\pi$, but the agent collects data using the behaviour policy $b$, which is, generally, a more exploratory policy. Since the distrubtion of the target and behaviour policies are different, we need a way to "correct" for the difference in distributions, which is importance sampling.

### The Mathematics
Suppose we want to estimate the expected value of some function $f(x)$ under a target distribution $p(x)$, but we only have samples from a different distribution $q(x)$. Then:

$$
\mathbb{E}_{x \sim p}[f(x)] = \sum_x p(x) f(x) = \sum_x \frac{p(x)}{q(x)} q(x) f(x) = \mathbb{E}_{x \sim q}\left[\frac{p(x)}{q(x)} f(x)\right]
$$

Here, $\frac{p(x)}{q(x)}$ is the importance sampling ratio, which reweights the samples to reflect their importance under the target distribution. The importance sampling ratio reflects the ratio between the two distributions, which allows us to estimate expectations under one distribution using samples from another by accounting for the difference in likelihood between the two.

### Importance Sampling in Reinforcement Learning
In the context of RL, the goal is to estimate the expected return under a target policy $\pi$, but the data (state-action-reward trajectories) was generated by a different behaviour policy $b$. Even if the actions taken differ, both the target policy $\pi$ and the behaviour policy $b$ interact with the same environment. This means the state transition dynamics are the same for both, since they are governed by the MDP and do not depend on the policy itself. As a result, the only difference between trajectories generated by $b$ and those from $\pi$ lies in how actions are selected at each state, which is the difference in the distributions of the policies. By accounting for this difference with importance sampling, we can accurately estimate the expected return of the target policy $\pi$ using samples generated by the behaviour policy $b$.

* Let $\pi(a_t \mid s_t)$ be the probability of taking action $a_t$ in state $s_t$ under the target policy $\pi$.
* Let $b(a_t \mid s_t)$ be the probability of taking the same action under the behaviour policy $b$.

At time step $t$, the importance sampling ratio is
$$\rho_t = \frac{\pi(a_t \mid s_t)}{b(a_t \mid s_t)}$$

This ratio tells us how to reweight a sample from the behaviour policy $b$ to estimate its expected contribution under the target policy $\pi$. In other words, it represents the ratio of the likelihood of taking the same action $a$ in the same state $s$ between the two policies.

* If $\rho_t$ is small, it means this action was much more likely under $b$ than under $\pi$, so this sample is less relevant to what $\pi$ would do. We downweight it.
* If $\rho_t$ is large, it means this action is more consistent with what $\pi$ would have done. We upweight it.

Essentially, $\rho_t$ tells us how much a sample from $b$ should count when estimating what $\pi$ would do. It adjusts for the differences of the distributions between the two policies, so that we can use data from one policy to estimate the behaviour of another.

To estimate the return under the target policy $\pi$ over a full trajectory of length $T$, we compute the cumulative importance sampling ratio from time $t$ onward, where

$$\rho_{t:T-1} = \prod_{k=t}^{T-1} \frac{\pi(a_k \mid s_k)}{b(a_k \mid s_k)} = \rho_t \cdot \rho_{t+1} \cdots \rho_{T-1}$$

This cumulative ratio adjusts for the differences between the target policy $\pi$ and the behaviour policy $b$ at each timestep along the trajectory. It effectively reweights the entire trajectory so that it reflects how likely the actions would be under the target policy. Then, given a trajectory generated by $b$, we can estimate the expected return under the target policy $\pi$ using:

$$v_\pi(s_t) = \mathbb{E}_b[\rho_{t:T-1} \cdot G_t \mid s_t] \approx \frac{1}{N} \sum_{i=1}^N \rho^{(i)} G^{(i)}$$

* $G_t$: the observed return from time $t$ onward based on the behaviour policy $b$, where $G_t = r_{t+1} + \gamma r_{t+2} + \gamma^2 r_{t+3} + \dots$
* $\rho_{t:T-1}$: the cumulative importance sampling ratio
* $\rho^{(i)} G^{(i)}$: one reweighted return from the $i$-th trajectory.
* $N$: the total number of trajectories sampled from the behaviour policy $b$.

So if we have multiple trajectories, we compute the adjusted return for each using $\rho^{(i)} G^{(i)}$, and then average them to estimate the expected return under the target policy.

## Off-Policy Monte Carlo Method
Off-policy Monte Carlo learning enables us to evaluate and improve a target policy $\pi$ using episodes generated from a different behaviour policy $b$. Both policies can be either deterministic or stochastic. The only essential requirement is the coverage condition as discussed before, where

$$\text{If } \pi(a \mid s) > 0, \text{ then } b(a \mid s) > 0$$

### Algorithm Steps
1. Initialize:
   - Initialize a target policy $\pi$ (can be random).
   - Initialize the action-value function $q(s, a)$ arbitrarily.

2. Generate an Episode:
   - Follow the behaviour policy $b$ to generate an episode.
   - Record the sequence of transitions: $(s_0, a_0, r_1), (s_1, a_1, r_2), \dots, (s_T, \_)$

3. Policy Evaluation (via Importance Sampling):
   - For each timestep $t$ in the episode, process in reverse from $T-1$ to $t$:
     - Compute the return $G_t$ from time $t$ onward: $G_t = r_{t+1} + \gamma r_{t+2} + \gamma^2 r_{t+3} + \dots$
     - Compute the cumulative importance sampling ratio: $\rho_{t:T-1} = \prod_{k=t}^{T-1} \frac{\pi(a_k \mid s_k)}{b(a_k \mid s_k)}$
     - Estimate the reweighted return: $v_\pi(s_t) \approx \rho_{t:T-1} \cdot G_t$
     - Use this to update the estimate of $q(s_t, a_t)$
     - Average over multiple episodes (if applicable): $v_\pi(s) \approx \frac{1}{N} \sum_{i=1}^N \rho^{(i)} G^{(i)}$

4. Policy Improvement:
   - For every state $s$ visited in the episode:
     - Improve the policy using a greedy or $\varepsilon$-greedy update:
       $$\pi(s) = \arg\max_a q(s, a) \quad \text{or} \quad \varepsilon\text{-greedy}(q(s, a))$$

5. Repeat:  
   - Go back to Step 2 and repeat the process until the policy converges or is "good enough."

# Temporal Difference (TD) Learning
Temporal Difference (TD) learning is a fundamental reinforcement learning method that blends the strengths of Dynamic Programming (DP) and Monte Carlo (MC) methods.

* Like Monte Carlo, TD learning can learn directly from experience, using the sequences of states, actions, and rewards without requiring a model of the environment’s dynamics.
* Like Dynamic Programming, TD learning uses bootstrapping, meaning it updates value estimates using other learned estimates rather than waiting for the final outcome of an episode.

This means TD learning can update the value of a state immediately after the next state is observed, making it more efficient and suitable for online, incremental learning, even in continuing (non-episodic) tasks.

## TD Prediction
TD learning also learns from experience, which are the trajectories generated by following a policy $\pi$. In MC, the value of a state is updated using the full return from that state until the end of the episode

$$v(S_t) \leftarrow v(S_t) + \alpha \left[ G_t - v(S_t) \right]$$

where $G_t$ is the return observed from time step $t$ to the end of the episode. Since this return is computed recursively from the full episode, MC must wait for the episode to finish before updating any values. In contrast, TD learning uses bootstrapping. Instead of waiting for the full return, it updates the value of a state using the observed immediate reward and the estimated value of the next state, where

$$v(S_t) \leftarrow v(S_t) + \alpha \left[ R_{t+1} + \gamma v(S_{t+1}) - v(S_t) \right]$$

This means at each time step, TD can immediately update the previous state’s value using the reward and the estimated value of the next state, without waiting for the episode to complete.

The term  
$$\delta_t = R_{t+1} + \gamma v(S_{t+1}) - v(S_t)$$
is called the TD error, which is the difference between the estimated return based on the new sample and the previous estimate. This algorithm is known as TD(0) or one-step TD.


### TD(0) Algorithm for Policy Evaluation
1. Given:
   - A policy $\pi$ to evaluate.
   - A learning rate (step size) $\alpha \in (0,1]$.
   - A discount factor $\gamma \in [0,1]$.

2. Initialize:
   - Value function $v(s)$ arbitrarily for all $s \in \mathcal{S}$

3. Repeat (over episodes):
   - Initialize starting state $S_0$
   - For each timestep $t$ in the episode:
     - Choose action $A_t \sim \pi(\cdot \mid S_t)$
     - Observe reward $R_{t+1}$ and next state $S_{t+1}$
     - Update the value of the current state:
       $$v(S_t) \leftarrow v(S_t) + \alpha \left[ R_{t+1} + \gamma v(S_{t+1}) - v(S_t) \right]$$

4. Continue:
   - Repeat episodes until convergence of $v(s)$

## Advantages of TD
1. Model-Free: TD methods do not require a model of the environment’s dynamics (transition probabilities or reward function), making them applicable to unknown or complex environments.

2. Online and Incremental Updates: TD can update value estimates after every step without waiting for the episode to finish. This is especially beneficial for problems with long episodes or continuing (non-terminating) tasks.

3. Faster Convergence in Stochastic Environments: In many stochastic tasks, TD methods tend to converge faster and with less variance than Monte Carlo methods, since they bootstrap from existing estimates rather than relying solely on complete returns so it is more sample efficient.

<img src="https://towardsdatascience.com/wp-content/uploads/2023/08/1kGWG1Z3TxHg-dGbZ7DLR1g.png" width=500>

## Bias and Variance Tradeoff
Bias refers to the systematic error between the agent's target estimation and the true value. Variance refers to how much the target estimation fluctuates across different episodes for the same state.

* Monte Carlo (Low Bias, High Variance): MC relies on the actual total return ($G_t$) from a completed episode. Because the target is calculated using real, completed experience, it is an unbiased estimate of the true value. However, the variance is high because the return depends on the accumulated randomness of every single action and state transition in the entire episode. One lucky or unlucky step early on can drastically swing the final total. As you collect more samples, the average of these high-variance samples will eventually converge to the true value.

* TD (High Bias, Low Variance): TD relies on bootstrapping, meaning it updates its estimate using its own existing guess ($V(s)$) rather than the real final outcome. Because the existing guess is likely imperfect (especially at the start), the target is systematically incorrect, leading to high bias. The high bias is generally not a problem, but under certain setting the estimation might diverge. However, the variance is low because the target depends on only one step of randomness (the immediate next state and reward), ignoring the randomness of the rest of the episode.
   
## n-Step TD Algorithm
n-Step TD is the generalization of TD(0) and Monte Carlo. While TD(0) updates based on just the immediate next step, n-Step TD looks $n$ steps into the future before bootstrapping.

Therefore, the n-step Return ($G_{t:t+n}$) is calculated as the sum of the first $n$ rewards plus the estimated value of the state $n$ steps later

$$G_{t:t+n} = R_{t+1} + \gamma R_{t+2} + \dots + \gamma^{n-1} R_{t+n} + \gamma^n V(S_{t+n})$$

And the Value Update Rule is

$$V(S_t) \leftarrow V(S_t) + \alpha \left[ G_{t:t+n} - V(S_t) \right]$$

By increasing the value of $n$, the algorithm looks further into the future to use more observed real rewards ($R$) instead of relying heavily on the estimation of the next state's value ($V$).
* As $n$ increases: The Bias decreases (closer to real data), but the Variance increases (more random steps accumulated).
* As $n$ decreases: The Bias increases (more bootstrapping), but the Variance decreases.

Conceptually, Monte Carlo (MC) is essentially TD($\infty$), where the agent looks infinite timesteps into the future (or until the episode terminates), using only real rewards and no bootstrapping.

In n-Step TD algorithms, the parameter $n$ must be tuned carefully to find the "sweet spot" in the error curve:
1. Initial Improvement: As $n$ increases from 1, the prediction error usually drops because the bias is strictly decreasing. We are relying less on the potentially faulty initial value estimates.
2. The Turning Point: However, if $n$ becomes too large, the error starts to increase again. At this point, the bias is already very low, but the variance begins to dominate. The target becomes too noisy because it includes too many random transitions.

Therefore, the optimal $n$ is usually an intermediate value that balances the stability of TD with the accuracy of MC, which means balancing the bias and variance within the estimations.

<img src="https://lcalem.github.io/imgs/sutton/nstep_graph.png" width=700>

## TD($\lambda$)
n-Step TD looks exactly n steps ahead and then applies bootstrapping. However, this method requires picking a specific $n$ to balance bias and variance, which is brittle and difficult to tune. TD($\lambda$) solves this by using all available n-step horizons simultaneously. It calculates a weighted average of all n-step returns. This approach utilizes more data and replaces the hard choice of a specific $n$ with a smoother parameter $\lambda$. The weighting follows an exponential decay, placing more emphasis on shorter n-step returns (targets closer to the current state). This ensures that immediate, lower-variance estimates contribute most to the update, while longer-term returns still help correct bias.

For a timestep $t$ in a trajectory, we define its n-step return ($G_t^{(n)}$) to be a target constructed by summing actual rewards for $n$ steps, and then bootstrapping (estimating) the rest of the value function.

* 1-Step Return (TD(0)):
    $$G_t^{(1)} = R_{t+1} + \gamma V(S_{t+1})$$
    * Estimate the current states with the estimation of the next state. High bias, low variance.

* 2-Step Return:
    $$G_t^{(2)} = R_{t+1} + \gamma R_{t+2} + \gamma^2 V(S_{t+2})$$
    * Use more real reward from experiment and less estimation of next state. Decreasing bias, increasing variance.

* Infinite-Step Return (TD($\infty$) = Monte Carlo):
    $$G_t^{(\infty)} = R_{t+1} + \gamma R_{t+2} + ... + \gamma^{T-1} R_T$$
    * Use pure reward signal without estimation of another state. Zero bias, high variance.

n-step TD will only select one of the step to use, but TD($\lambda$) uses all of the n-step returns to calculate a weighted average of every possible horizon. We define a new target called the $\lambda$-Return ($G_t^\lambda$), where

$$
\begin{aligned}
G_t^\lambda &= (1 - \lambda) \sum_{n=1}^{\infty} \lambda^{n-1} G_t^{(n)} = (1 - \lambda) \cdot \underbrace{\left( R_{t+1} + \gamma V(S_{t+1}) \right)}_{\text{1-step return } (G_t^{(1)})} 
+ (1 - \lambda)\lambda \cdot \underbrace{\left( R_{t+1} + \gamma R_{t+2} + \gamma^2 V(S_{t+2}) \right)}_{\text{2-step return } (G_t^{(2)})} + (1 - \lambda)\lambda^2 \cdot \underbrace{\left( R_{t+1} + \gamma R_{t+2} + \gamma^3 R_{t+3} + \gamma^3 V(S_{t+3}) \right)}_{\text{3-step return } (G_t^{(3)})}
+ \dots
\end{aligned}
$$

* $G_t^{(n)}$: the n-step return for the a state at timestep $t$.
2. $\lambda$: the decay factor between 0 and 1, so as $n$ increases (looking further into the future), the weight we assign to that return decays because we care more about the more recent returns.
3. *$(1 - \lambda)$: a normalization constant that ensures that all the weights sum up to exactly 1, so the final target is mathematically valid.

The parameter $\lambda$ acts as a slider for how much you trust "deep" future data versus your current immediate estimate. A smaller $\lambda$ means putting more empahsis onto the recent data, where a larger $\lambda$ means putting more weights onto the future data.

* If $\lambda = 0$ (TD-0):
    The formula collapses. The weight for the 1-step return becomes 1, and all other weights become 0, so only the immediate next step is considered, which is equivalent to TD-0.

* If $\lambda = 1$ (Monte Carlo):
    Impossbile because when $\lambda = 1$, the return is just 0. However, as $\lambda$ approaches 1, the decay disappears. The algorithm puts all the weight on the longest possible return (the full episode), so only the actual final outcome matters, which is equivalent to Monte-Carlo

* If $0 < \lambda < 1$ (TD Lambda):
    Balanced approach, where the immediate steps have the highest weight (low variance) while the distant steps still contribute, correcting the bias of your initial estimates.

The algorithm at each timstep uses all valid $n$-step returns available from the current state until the terminal state, so the earlier timestep will have more data avalible and timestep closer to the terminal state will have less data avaliable.

* At the beginning of the trajectory, the agent looks ahead at 1-step, 2-step, 3-step... all the way to the final step $T$ (the Monte Carlo return) and calculates a weighted average of all these returns, which blends the low-variance bias of TD(0) with the high-variance unbiasedness of Monte Carlo.

* At the end of the trajectory, the agent does not have any future steps. At step $T-1$, the only valid return is the 1-step return ($R_T + \gamma V(S_T)$), which effectively behaving like TD(0).


### The Forward View
The reason above explanation is often separated from implementation is because this is the forward view of the algorithm, so in order to compute $G_t^{(n)}$, we need to wait for the entire episode to terminate, making it impossible perform online learning.

<img src="https://towardsdatascience.com/wp-content/uploads/2019/08/1L-LUOyW5W-0gBxx80GHdHQ.png" width=500>

## TD for Control
When using TD learning for control, knowing the optimal state-value function from policy evaluation alone is not enough in a model-free setting, because we do not know the environment’s transition dynamics. Without a model, we can’t directly compute which action leads to the best next state from just $v(s)$. Instead, we learn the action-value function $q(s, a)$, which estimates the expected return for taking action $a$ in state $s$ and then following the policy. This allows us to both evaluate and improve the policy directly. We will address 3 different TD control algorithms: SARSA, Q-learning, and expected SARSA


### SARSA (On-Policy TD Control)
SARSA is an on-policy TD control algorithm that learns the optimal action-value function. It is similar to the TD(0) method for state-value estimation, but instead of updating from state to state, it updates from state-action pair to state-action pair, where

$$q(S_t, A_t) \leftarrow q(S_t, A_t) + \alpha \left[ R_{t+1} + \gamma \, q(S_{t+1}, A_{t+1}) - q(S_t, A_t) \right]$$

This update requires the five key events:
- $(S_t, A_t)$: current state-action pair
- $R_{t+1}$: reward from taking $A_t$ in $S_t$
- $(S_{t+1}, A_{t+1})$: next state-action pair chosen according to the current policy

Because SARSA uses $A_{t+1}$ from the current policy, it learns the value of the policy as it is being followed (on-policy learning). After each update, the policy can be improved by making it greedy or $\epsilon$-greedy with respect to the updated $q(s, a)$.

### SARSA Algorithm
1. Given:
   - Initial policy $\pi$ (e.g., $\epsilon$-greedy)
   - Learning rate $\alpha \in (0, 1]$
   - Discount factor $\gamma \in [0, 1]$

2. Initialize:
   - $q(s, a)$ arbitrarily for all $s \in \mathcal{S}$, $a \in \mathcal{A}$

3. Repeat (for each episode):
   - Initialize starting state $S_0$
   - Choose action $A_0 \sim \pi(\cdot \mid S_0)$
   - For each timestep $t$:
     - Take action $A_t$, observe the reward $R_{t+1}$ and the next state $S_{t+1}$
     - Choose the next action according to the current policy $A_{t+1} \sim \pi(\cdot \mid S_{t+1})$
     - Update:
       $$q(S_t, A_t) \leftarrow q(S_t, A_t) + \alpha \left[ R_{t+1} + \gamma \, q(S_{t+1}, A_{t+1}) - q(S_t, A_t) \right]$$
     - Update $\pi$ to be greedy or $\epsilon$-greedy with respect to $q$

4. Repeat until convergence:
   - Continue until $q(s, a)$ converges.

Note: MC methods update only after an episode finishes. If an agent becomes trapped in a state (or a loop), the episode may never end, preventing any learning from occurring. In contrast, online learning methods such as SARSA update their estimates during the episode at every timestep. This allows the agent to quickly recognize that a policy leading to such traps is poor and adjust its behavior before the episode is over.

## Q-Learning
Q-learning is an off-policy TD control algorithm that directly learns the optimal action-value function $q_*$, rather than the action-value function of the current policy $q_\pi$. This differs from SARSA (on-policy), which learns $q_\pi$ for the policy being followed. The Q-learning update rule is

$$q(S_t, A_t) \leftarrow q(S_t, A_t) + \alpha \left[ R_{t+1} + \gamma \max_a q(S_{t+1}, a) - q(S_t, A_t) \right]$$

* The max operator selects the value of the best possible next action, regardless of the current policy.
* This makes Q-learning off-policy: it learns as if the agent is always acting greedily, even if it is exploring in practice.

Both SARSA and Q-learning are sample-based solutions to the Bellman equations and can converge to the true action-value functions given enough samples. However there's a key difference in which Bellman equation the algorithm is solving

- SARSA solves the Bellman expectation equation for $q_\pi$, where
  $$q_\pi(s, a) = \sum_{s'} \sum_{r} p(s', r \mid s, a) \left[ r + \gamma \sum_{a'} \pi(a' \mid s') q_\pi(s', a') \right]$$

- Q-learning solves the Bellman optimality equation for $q_*$, where
  $$q_*(s, a) = \sum_{s'} \sum_{r} p(s', r \mid s, a) \left[ r + \gamma \max_{a'} q_*(s', a') \right]$$


Therefore, SARSA need to follow policy iteration by first estimate $q_\pi$ from experience, then make the policy greedy (or $\epsilon$-greedy) with respect to $q_\pi$. On the other hand, Q-learning effectively uses value iteration. It updates $q(s,a)$ as if it were already the optimal $q_*$, combining policy evaluation and improvement in a single step during the update.

Q-learning is an off-policy algorithm because it always learns from the value of the best possible action it could take in the next state, rather than the value of the action it actually took. In this case, the target policy (the policy Q-learning is trying to learn) is always greedy with respect to the current action-value estimates, which is not needed during learning because we only aim to get the optimal action-value function, $q_*$. Once $q_*$ is obtained, we can directly extract the optimal policy from it by acting greedily with respect to $q_*$. The behaviour policy (the policy used to select actions during learning) can be any exploratory policy, such as $\epsilon$-greedy or even random, as long as it ensures sufficient exploration. This separation between the behavior policy and the target policy is what makes Q-learning off-policy. 

Although Q-learning is an off-policy algorithm, it does not require importance sampling to reweight trajectories. Importance sampling is typically used to correct for differences in action-selection probabilities between the target policy and the behavior policy. However, Q-learning’s update rule always evaluates the next state using the greedy action with respect to the current action-value estimates, regardless of the action actually taken by the behavior policy. Because the target in the update is determined solely by the current Q-values and not by the behavior policy’s action probabilities, no such correction is needed.

### Q-Learning Algorithm
1. Given:
   - Learning rate $\alpha \in (0, 1]$
   - Discount factor $\gamma \in [0, 1]$

2. Initialize:
   - $q(s, a)$ arbitrarily for all $s \in \mathcal{S}$, $a \in \mathcal{A}$

3. Repeat (for each episode):
   - Initialize starting state $S_0$
   - Choose initial action $A_0$ using an exploration policy (e.g., $\epsilon$-greedy based on $q$)
   - For each timestep $t$:
     - Take action $A_t$, observe reward $R_{t+1}$ and next state $S_{t+1}$
     - Update:
       $$q(S_t, A_t) \leftarrow q(S_t, A_t) + \alpha \left[ R_{t+1} + \gamma \max_a q(S_{t+1}, a) - q(S_t, A_t) \right]$$
     - Choose next action $A_{t+1}$ using the exploration policy

4. Until convergence:
   - Continue until $q(s, a)$ stabilizes or a predefined stopping condition is met.

## Expected SARSA  
Expected SARSA is another TD control algorithm that improves upon the standard SARSA method by uinsg the expected value of the next state's action-value (weighted by the policy's action probabilities) to update the current estimation. The update rule for expected SARSA is

$$q(S_t, A_t) \leftarrow q(S_t, A_t) + \alpha \left[ R_{t+1} + \gamma \sum_{a} \pi(a \mid S_{t+1}) \, q(S_{t+1}, a) - q(S_t, A_t) \right]$$  

Unlike SARSA, which updates its estimates using a single sampled action from the next state, Expected SARSA computes a weighted average over all possible actions in the next state, weighted by the policy's action probabilities. This reduces variance in the estimation process because SARSA's reliance on a single sample can lead to high variability, as the update depends heavily on whether the sampled action is favorable or not. In contrast, Expected SARSA’s use of the expected value produces a deterministic and more stable update, often allowing for larger step sizes ($\alpha$) without destabilizing learning.

The reason is that SARSA’s updates are influenced by the randomness of a single action choice, which can make large-step-size update overly aggressive and prone to divergence. Expected SARSA, on the other hand, bases its update on the expected return under the policy, smoothing out randomness across all possible actions. As learning progresses and the action-value estimates converge, the computed expected value approaches the current estimate, where $\sum_{a} \pi(a \mid S_{t+1}) \, q(S_{t+1}, a) \approx q(S_t, A_t)$, meaning the update term goes to zero and learning stabilizes after enought training.

<img src="https://www.researchgate.net/publication/224446259/figure/fig3/AS:667853514625027@1536240094240/Average-return-on-the-cliff-walking-task-over-the-first-n-episodes-for-n-100-and-n.png" width=500>

However, this reduction in variance comes at a computational cost. Since expected SARSA must evaluate and weight all possible actions at each timestep, it becomes computationally expensive, particularly in environments with large action spaces. Despite this trade-off, Expected SARSA is often preferred in scenarios where variance reduction is critical, and computational resources are sufficient to handle the additional overhead.

### On-Policy and Off-Policy Flexibility  
Expected SARSA can function as both an on-policy and off-policy algorithm, depending on how actions are selected during learning. The key to this flexibility lies in its update rule, which computes the expected action-value over all possible next actions, weighted by the policy $\pi(a \mid S_{t+1})$. This update computation is independent of the actual action taken, meaning expected SARSA can function as both an on-policy and off-policy algorithm

* On-policy case: If the policy used to select actions (e.g., $\epsilon$-greedy) is the same as the policy being evaluated ($\pi$), Expected SARSA operates on-policy. The expectation aligns with the agent's current behavior policy.  
* Off-policy case: If actions are selected using a different policy (e.g., a purely exploratory policy) while the expectation is computed under a target policy ($\pi$), the algorithm becomes off-policy.  

Note: Q-learning is a special case of Expected SARSA where the target policy is deterministic, always selecting the action with the highest $q$-value (i.e., $\pi(a \mid S_{t+1}) = 1$ for $a = \arg\max_a q(S_{t+1}, a)$). This makes Q-learning inherently off-policy, as it learns the optimal policy while potentially following an exploratory policy, making it an special case of Expected SARSA

## Summary
| Algorithm        | On/Off Policy | Update Target   | Key Idea                                                                                          | Pros                                                                                              | Cons                                                                                              |
|------------------|--------------|-------------------------------------------------------------------------------|---------------------------------------------------------------------------------------------------|---------------------------------------------------------------------------------------------------|---------------------------------------------------------------------------------------------------|
| **SARSA**        | On-policy    | $$R_{t+1} + \gamma \, q(S_{t+1}, A_{t+1})$$ | Updates using the *actual* action taken by the current policy.                                    | Simple; naturally incorporates exploration in updates.                                           | Higher variance (depends on a single sample); may require smaller step sizes for stability.      |
| **Q-learning**   | Off-policy   | $$R_{t+1} + \gamma \, \max_{a} q(S_{t+1}, a)$$ | Updates towards the *best possible* action value, regardless of the action actually taken.        | Directly learns the optimal policy; converges faster in some settings.                           | Can overestimate action values; requires good exploration policy to cover state-action space.    |
| **Expected SARSA** | Both         | $$R_{t+1} + \gamma \, \sum_a \pi(a \mid S_{t+1}) \, q(S_{t+1}, a)$$ | Updates using the *expected* value over all possible actions under the policy.                    | Lower variance; more stable updates; can use larger step sizes without harming convergence.      | Higher computation cost (must compute weighted sum over all actions at each step).               |

# Models In Reinforcement Learning
In reinforcement learning, a model stores knowledge about the environment dynamics, which tells the agent how the environment responds to actions by producing rewards and transitioning to new states. In other words, the model tells the agent how the world works based on its current understanding. By using this knowledge, the model can be used to simulate the environment and produce simulated experience for the agent to learn from. There are two types of models
1. Sample models: return a single possible outcome by sampling from the underlying probability distribution of state transitions and rewards.
    - Pros: Fast, memory-efficient.
    - Cons: Less accurate, as each sample is only an approximation of the full distribution.

2. Distribution models: specify the complete probability distribution over all possible outcomes.
    - Pros: More accurate and informative.
    - Cons: Larger, more computationally expensive.
    - Note: A distribution model can also be used to generate a sample model by sampling from it.

In reinforcement learning, there are two types of learning methods, model-based and model-free methods. Model-based methods. Model-based methods rely heavily on planning—using the model to simulate possible futures, evaluate them, and improve the policy before acting in the real environment, like DP, which looks ahead for the future rewards before selecting an action. Model-free methods, on the other hand, do not learn or estimate the model and learn the value function directly from experience through trial and error, like MC or TD methods.

Note: For learning to happen, the agent must interact with an environment, which itself follows some transition and reward dynamics (this is the “true model” of the world). The key difference between model-based and model-free methods lies in whether the learning algorithm explicitly learns or uses its own approximation of this model. Both approaches aim to improve estimates of value functions (and ultimately the policy), but model-based methods do so by first estimating the transition and reward functions and then planning with them, whereas model-free methods skip this step and learn value functions directly from experience without building an explicit model.

In any reinforcement learning setup, there is always one model that truly exists, which is the environment’s real rules. This is the true model of the world, whether it’s physics in the real world or the programmed logic in a simulator. The difference between model-based and model-free methods is about whether the agent also builds a second model of its own:

Model-based methods have two models in play
* The true model: the real environment’s transition and reward dynamics that the agent interact with.
* The agent’s learned model: its internal guess at how the environment works, which is used for planning and action selection

The agent first tries to learn this internal model from experience. Once it has it, it can run mental simulations inside its head, planning several steps ahead without touching the real environment.

Model-free methods have only one model
* The true environment model that the agent interact with

The agent doesn’t bother creating its own copy. Instead, it directly learns value functions and policies from trial-and-error experience, without explicitly modeling how states change or what rewards to expect.

# Planning with Model Experience
In model-based RL, planning takes the model as input and outputs an improved policy for interacting with the environment. The process can also refine the model itself as more real-world data is gathered, creating a continuous loop of improvement between model learning and policy optimization. The key of planning is to obtain a better estimation of the value functions

## Random Tabular Q-Planning
Random tabular Q-planning is a model-based reinforcement learning algorithm. Instead of interacting with the real environment, it uses a learned or known model of the environment to simulate experiences and update the action-value function. At each step, the algorithm picks a random state–action pair (not necessarily from a real trajectory), queries the model for the resulting reward and next state, and then performs a Q-value update.  
By simulating many such updates from the model, the policy can improve without needing to gather new real-world experience, follows the algorithm below

1. Initialize:
   - For all $ s \in \mathcal{S} $ and $ a \in \mathcal{A} $, set $ q(s, a) $ arbitrarily (e.g., 0).

2. Loop (Planning):
   - Randomly select a state $ S_t $ and an action $ A_t $ from the state–action space.
   - Use the model to get the predicted reward $ R_{t+1} $ and next state $ S_{t+1} $. (If using a distribution model, you may sample or use expected values.)
   - Update:
     $$
     q(S_t, A_t) \leftarrow q(S_t, A_t) + \alpha \left[ R_{t+1} + \gamma \max_a q(S_{t+1}, a) - q(S_t, A_t) \right]
     $$
   - Optionally, update the policy to be greedy or $\epsilon$-greedy with respect to $ q $.


Note: All updates in this algorihtm use simulated experience from the model, not real-world interaction.

## Dyna-Q
Dyna-Q is an architecture that allows the agent to perform online planning by integrating learning, planning, and acting into a single unified process, allowing the agent to interact with the real environment while simultaneously planning for future steps. In online planning, each real experience is used in two ways:
1. Direct Reinforcement Learning (Direct RL): The agent immediately updates its value function and improve the policy using the real experience.
2. Model Learning: The agent uses the real experience to improve its internal model of the environment’s dynamics, making it more accurately match the real world for planning purposes.

By maintaining a model through model learning, the agent can generate simulated experiences to improve the policy without further real-world interaction. This makes fuller use of limited data, enabling better performance with fewer environmental interactions. On the other hand, direct RL is simpler and free from biases that may arise from an imperfect model since it updates only from actual outcomes.

<img src="https://miro.medium.com/v2/resize:fit:554/1*j5chypwunyX00ejvuu10Gw.png" width=300>

Dyna-Q runs the following processes continuously and in parallel:
1. Planning through simulated experiences generated by the model
2. Acting by interacting with the environment  
3. Model learning by updating the internal model from real experiences
4. Direct RL through updating the value function from real experiences

For direct RL, Dyna-Q uses one-step tabular Q-learning like before, where
$$q(S_t, A_t) \leftarrow q(S_t, A_t) + \alpha \left[ R_{t+1} + \gamma \max_a q(S_{t+1}, a) - q(S_t, A_t) \right]$$

For planning, Dyna-Q uses random-sample one-step tabular Q-planning, which consists of:
1. Search Control: Randomly select a state–action pair from those the agent has previously experienced. If the agent has never visited a state–action pair, it cannot plan from it because no outcome information is available.
2. Model Query: Feed the selected state–action pair into the model, which returns a simulated reward and next state. In the basic Dyna-Q formulation, the model is assumed deterministic, meaning it returns the same result for a given state–action pair as last observed.
3. Value Update: Apply the same Q-learning update rule as in direct RL, but using the simulated transition as if it were real.

Note: Each Q-planning step simulates and updates only one state–action pair (a single one-step transition) at a time. However, Q-planning is performed many times in succession after every real Q-learning update, meaning the agent will plan for multiple times before it takes every action in real life.

Both Q-learning (direct RL) and Q-planning share and update the same action-value function. The difference lies in the source of the experience, where direct RL updates are based on real interactions with the environment and Q-planning updates are based on simulated interactions using the agent’s learned model. In this sense, Q-planning is essentially Q-learning in simulation, meaning Dyna-Q does not reduce the computational effort per update (it actually adds a bit more computation) since planning requires querying and simulating with the model.

However, Dyna-Q greatly improves sample efficiency. By reusing real experiences through the model, the agent can perform many extra updates in simulation without needing to gather new real-world data. This allows it to learn a good policy with fewer real episodes, which is especially valuable when real interactions are costly or limited.

<img src="https://www.mdpi.com/drones/drones-06-00365/article_deploy/html/images/drones-06-00365-g006-550.jpg" width=500>

One limitation of Dyna-Q is its inefficiency in the early stages of learning. When the agent has very limited or no data, most planning steps are not meaningful because the model has little information to simulate with. As a result, many of the early updates contribute little to improving the policy. However, once the agent begins to collect real experiences, planning can replay these experiences in simulation, compounding their value and allowing the agent to improve with far fewer real interactions. This makes Dyna-Q particularly powerful in later phases of training.

Another limitation lies in the search control step. In Dyna-Q, state–action pairs are sampled at random for planning, which can be inefficient, especially when the state–action space is very large. Many updates may focus on unimportant or rarely visited states, leading to wasted computation. In such cases, a large number of planning steps is required before the updates significantly impact performance.

### Dyna-Q Algorithm

1. Initialize:
   - For all $s \in \mathcal{S}$ and $a \in \mathcal{A}$, set $q(s,a)$ arbitrarily (e.g., $0$).
   - Initialize an empty model $\mathcal{M}$ (e.g., a table mapping $(s,a)\!\to\!(r,s')$).
   - Set hyperparameters: learning rate $\alpha$, discount factor $\gamma$, exploration rate $\epsilon$, and planning steps $n$.

2. Loop (for each episode or until stopping):
   - Initialize start state $S$.

       (A) Acting + Direct RL
       - Choose action $A$ using an exploration policy (e.g., $\epsilon$-greedy).
       - Take $A$ in the real environment; observe $R$ and next state $S'$.
       - Q-learning update (direct RL):
         $$q(S,A) \leftarrow q(S,A) \;+\; \alpha \left[ R \;+\; \gamma \max_{a} q(S',a) \;-\; q(S,A) \right]$$
       - Model learning (record the transition):
         - Store/update $\mathcal{M}(S,A) \leftarrow (R,S')$.
       - Set $S \leftarrow S'$.

       (B) Planning (repeat $n$ times):
       - Search control: Randomly select a previously observed state–action pair $(\tilde S,\tilde A)$ from the keys of $\mathcal{M}$.
       - Model query: retrieve $(\tilde R,\tilde S') \leftarrow \mathcal{M}(\tilde S,\tilde A)$
         (for stochastic models, either sample or use expected values).
       - Q-planning update (simulated): 
         $$q(\tilde S,\tilde A) \leftarrow q(\tilde S,\tilde A) \;+\; \alpha \left[ \tilde R \;+\; \gamma \max_{a} q(\tilde S',a) \;-\; q(\tilde S,\tilde A) \right]$$

3. Repeat Step 2 until convergence or a stopping criterion is met (e.g., max episodes, small change in $q$).

<img src="https://miro.medium.com/1*a1ReZc7DyscMyo8nQpWuwg.png" width=400>

## Wrong Models
Planning in model-based RL depends entirely on the agent’s internal model of the environment. However, this model can often be wrong, meaning the stored transitions differ from what actually happens in the real environment. There are two main ways a model can be wrong:

1. Incomplete Model: The agent has not yet experienced certain state–action pairs, so their outcomes are simply missing from the model. This typically happens in the early stages of learning, when exploration is still limited. 

2. Inaccurate Model: The transitions stored in the model no longer match the environment’s true dynamics. This occurs most often when the environment is non-stationary (changing over time). In this case, planning may actually harm learning: the agent updates its value function based on outdated or incorrect information, pushing the policy in the wrong direction.  

### Addressing Wrong Models
1. Dealing with Incomplete Models: This can be easily solved by using an exploratory policy. With enough experience, the agent will eventually visit all state–action pairs, filling in the missing parts of the model and making it complete.

2. Dealing with Inaccurate Models: This requires continuous exploration to ensure the model stays up to date. If the environment changes, the only way to correct the model is to revisit those states and actions. A common strategy is **Dyna-Q+**, an algorithm that adds an exploration bonus to the reward to encourage the agent to revisit states that have not been seen for a long time in real life, where

$$R = r + \kappa \sqrt{\tau}$$

* $r$: the original reward 
* $\kappa$: a small positive constant (bonus weight)  
* $\tau$: the number of time steps since the state was last visited
* $\sqrt{\tau}$: a bonus, artificial reward to encourage exploration

The artificial reward (exploration bonus) is used only within the agent’s internal model, not in the real environment. It grows over time for states that have been neglected to encourage the agent to revisit less-frequented states. Initially, this would update the action-value function in the wrong direction during planning since the bonus reward doesn’t actually exist in the real environment. But this distortion is intentional because during planning, the action-value function is biased upward for underexplored states. This makes those states look more attractive, encouraging the agent to select them in real interactions. Then, once the agent visits the state in real experience, it receives the true reward from the environment. The action-value function is updated again, correcting any bias introduced by the artificial bonus. In the long run, this process ensures that the value function converges to reflect the true environment dynamics, while the exploration bonus serves only as a temporary motivator. The result is a policy that balances exploration and exploitation, maintaining accuracy while improving sample efficiency.

# Reinforcement Learning with Function Approximation
So far, all the methods we have discussed rely on tabular representations, where each state or state–action pair has its own entry in a table storing its value. This works for small problems but becomes completely impractical when the state or action space grows large or infinite. In such cases, storing a value for every possible state or state–action pair would require infinite memory and computation, making tabular methods infeasible. To address this, we shift to function approximation. Instead of maintaining an explicit table, we use a parameterized function to approximate the value function. This allows us to represent an enormous set of states using a finite set of parameters. The goal is to find an approximation that is “good enough” given limited data, memory, and compute resources.

Function approximation reframes value estimation as a supervised learning problem: given a state or state–action pair, the function predicts its value, and the agent updates this function based on observed returns. This introduces the concept of generalization: leveraging patterns learned from past experiences to predict values for unseen states. Generalization significantly improves sample efficiency and scalability because the agent doesn’t need to experience every state individually. Instead, it can make informed decisions in unfamiliar states based on what it has learned from similar ones, enabling RL to work in complex, real-world environments where tabular methods are infeasible.

In contrast to generalization, which enables an agent to apply knowledge from similar states to new situations, discrimination is about deciding when two situations are not similar enough to share the same knowledge. Discrimination ensures that the agent treats states appropriately when they differ in ways that matter for decision-making. For example, tabular methods are an extreme case of perfect discrimination, where each state is treated as entirely unique and independent.

However, both extremes can lead to problems:
* Over-generalization occurs when the agent assumes that states are more similar than they actually are, leading to poor approximations and oversimplified decision-making.
* Over-discrimination happens when the agent treats even very similar states as completely different, which dramatically increases memory and computation requirements and slows learning.

In function approximation, the key is to find a balance between generalization and discrimination. The agent must know when to leverage previously learned information and when to treat new situations independently, ensuring efficient learning without sacrificing accuracy.

## On-Policy Prediction with Approximation
On-policy prediction with approximation means we learn a parameterized function, $\hat{v}(s; w)$, that estimates the true state-value function $v_{\pi}(s)$, such that $\hat{v}(s; w) \approx v_{\pi}(s)$. In the parametrized function, $s$ is the current state and $w$ is the weight vector of the function. The parameterized function, $\hat v$, can be a linear function, a non-linear function, or even a neural network.

Unlike tabular methods, where each state has its own entry, function approximation uses a finite set of weights to represent a potentially infinite number of states. Typically, the number of weights is much smaller than the total number of states. This has two major implications:

* Generalization: Updating one weight affects many states at once because the same parameters influence multiple predictions.
* Scalability: The method can handle large or continuous state spaces, where storing values for every state would be infeasible.

This approach allows reinforcement learning to move beyond small, discrete environments and operate effectively in complex real-world settings.

### Function Approximation as Supervised Learning
Function approximation in reinforcement learning can be viewed as a supervised learning problem where the input is a state $s$ and the output is the predicted value of that state, $v_{\pi}(s)$. The goal is to make these predictions as close as possible to the true returns under policy $\pi$. However, applying supervised learning methods directly in RL is not straightforward because RL presents unique challenges.

Unlike traditional supervised learning, which relies on a fixed dataset, reinforcement learning is an ongoing process that requires online learning, the ability to adapt continually as new experiences arrive. Furthermore, methods like TD learning and DP introduce additional complexity through bootstrapping, where the algorithm uses its current value estimates to predict future returns. This creates non-stationary targets because as the approximator updates, the targets themselves keep changing. Such instability can lead to divergence if the function approximation method is not designed to handle these conditions. These challenges highlight why not all function approximators are suitable for RL. Techniques must allow incremental updates, remain stable under bootstrapping, and cope with non-stationary data.

### Prediction Objective: Mean Squared Value Error
The prediction objective, denoted as $\bar{VE}$, measures how close our approximated value function $\hat{v}(s; w)$ is to the true value function $v_{\pi}(s)$ by computing the mean squared value error. In function approximation, adjusting the weights to improve accuracy for one state often affects predictions for other states due to shared parameters. Therefore, the goal is to minimize this objective so that the approximation performs well across all states.

However, not all states are equally important. Some states occur more frequently or have higher significance in decision-making. To account for this, we introduce a state distribution $\mu(s)$, which represents the relative importance (or probability) of each state. The weighted prediction objective is then defined as

$$\bar{VE} = \sum_{s \in \mathcal{S}} \mu(s) \big( v_{\pi}(s) - \hat{v}(s; w) \big)^2$$

* $\mu(s)$: A probability distribution over all possible states that indicates the importance or visitation frequency of each state $s$.
* $v_{\pi}(s)$: True value of state $s$ under policy $\pi$.
* $\hat{v}(s; w)$: Approximate value produced by the parameterized function.

Minimizing $\bar{VE}$ ensures that the approximation prioritizes accuracy for frequently encountered or critical states, resulting in better performance under realistic conditions.

### Gradient Descent for Value Function Approximation

To minimize the objective, we apply gradient descent, updating the weight vector $w$ in the direction of the negative gradient. Using a step size $\alpha$, the update rule is

$$w \leftarrow w - \alpha \frac{\partial \bar{VE}}{\partial w}$$

For a single state $s$, the gradient of the squared error is:

$$\frac{\partial}{\partial w} \big( v_{\pi}(s) - \hat{v}(s; w) \big)^2
= -2 \big( v_{\pi}(s) - \hat{v}(s; w) \big) \frac{\partial \hat{v}(s; w)}{\partial w}$$

So the weight update becomes:

$$w \leftarrow w + \alpha \big( v_{\pi}(s) - \hat{v}(s; w) \big) \nabla_w \hat{v}(s; w)$$

where $\nabla_w \hat{v}(s; w)$ is the gradient of the approximation function with respect to its weights. This term can be calcualted as long as the approximation function is differentiable almost everywhere, even if it is a complex neural network.


Notice that the gradient of the overall objective is:

$$
\nabla_w \bar{VE} = \sum_{s \in \mathcal{S}} \mu(s) \big( v_\pi(s) - \hat{v}(s; w) \big) \nabla_w \hat{v}(s; w),
$$

which requires summing over all states in the state space. This is infeasible for large or continuous spaces because computing this exact gradient would require enumerating every possible state.

To address this, we use stochastic gradient descent by approximating the full gradient using a single sampled state, where

$$
\bar{VE}(w) \;\approx\; \big(v_\pi(S_t) - \hat{v}(S_t;w)\big)^2.
$$

where $S_t$ is a state sampled from the distribution $\mu(s)$. In practice, however, the true value function $v_{\pi}(s)$ is unknown. To address this, we replace it with an empirical target based on experience based on this sample, where

$$\text{Target: } G_t = r_{t+1} + \gamma r_{t+2} + \dots$$

Since this update only uses one example from a sampled state, $G_t$, which is the sampled observed return of that state, is an unbiased but noisy estimate of that $v_{\pi}$. The SGD update then becomes

$$w \leftarrow w + \alpha \big( G_t - \hat{v}(S_t; w) \big) \nabla_w \hat{v}(S_t; w)$$

This update is computed for each observed state in real time, allowing the agent to learn online as new data arrives.

### Gradient Monte Carlo Algorithm
1. Input:
    - A policy $\pi$ for evaluation
    - A parametrized function $\hat v(s; w)$ that is used for estimation
    - Step size $\alpha$
2. Initialize all the weights $w$ arbitrarily
3. Loop
    - Generate an episode by following the policy $\pi$ and obtains a series {State, Action, Reward} 
    - Compute the return $G_t$ for each time step
    - For each time step, make an gradient update
        $$w \leftarrow w + \alpha \big( G_t - \hat{v}(S_t; w) \big) \nabla_w \hat{v}(S_t; w)$$
        
Notes: This algorithm uses Monte Carlo targets $G_t$, so it waits until the end of the episode to perform updates, which updates once for each time step in the episode after the episode finishes. This is not an online algorithm since the agent must wait for the episode to finish to learn.

### Semi-Gradient TD

The Gradient Monte Carlo method is unbiased but suffers from two main issues:  
1. It is slow to converge because it waits for full returns.  
2. It is not online, as it must wait until the episode ends to compute $G_t$.

We can address these issues by applying one-step bootstrapping with the TD algorithm. Instead of using the sampled return $G_t$ as the update target, we replace it with the TD target:

$$\text{TD target: } R_{t+1} + \gamma \hat{v}(S_{t+1}; w)$$

Thus, the weight update becomes:

$$w \leftarrow w + \alpha \big( R_{t+1} + \gamma \hat{v}(S_{t+1}; w) - \hat{v}(S_t; w) \big) \nabla_w \hat{v}(S_t; w)$$


This update does not compute the true gradient of the objective. This is because the TD target, $R_{t+1} + \gamma \hat{v}(S_{t+1}; w)$, itself depends on the weights $w$ through bootstrapping $\hat{v}(S_{t+1}; w)$, which can be changed with a change in weights. This introduces bias, because the update direction is not the true gradient. For this reason, the method is called semi-gradient TD.

The semi-gradient TD algorithm does not converge as robustly as gradient methods since it's gradient update is biased, but in most use cases, the convergence is good enough. Also, the semi-gradient TD algorithm has the following advantages
1. Online learning: Unlike Monte Carlo, this method can update after every step, not just after episodes.  
2. Faster convergence: Bootstrapping typically accelerates learning compared to pure Monte Carlo.  
3. Bias vs. variance: Semi-gradient TD introduces bias (due to ignoring part of the gradient), but it greatly reduces variance compared to MC, leading to faster practical convergence.


### Semi-Gradient TD Algorithm
1. Input:
    - A policy $\pi$ for evaluation
    - A parametrized function $\hat v(s; w)$ that is used for estimation
    - Step size $\alpha$
2. Initialize all the weights $w$ arbitrarily
3. Loop for each episode
    - For each timestep in the episode
        - Select action $A$ based on the policy
        - Take the action and observe the reward, $R$, and the transition to the next state $S_{t+1}$
        - Update the weights
          $$w \leftarrow w + \alpha \big( R_{t+1} + \gamma \hat{v}(S_{t+1}; w) - \hat{v}(S_t; w) \big) \nabla_w \hat{v}(S_t; w)$$
          
### Linear Methods
Linear approximation is a special and simpler case of general function approximation in reinforcement learning. Instead of directly using raw states, each state s is mapped to a real-valued feature vector

$$x(s) = [x_1(s), x_2(s), x_3(s), \dots, x_n(s)]$$

where the feature vector $x(s)$ has the same dimensionality as the weight vector $w$. The state-value function can then be approximated as a weighted sum (dot product) of these features

$$\hat{v}(s; w) = w^\top x(s) = \sum_{i=1}^n w_i , x_i(s)$$

For linear functions, the gradient with respect to the weight vector is straightforward, where
$$\nabla_w \hat{v}(s; w) = x(s)$$

This means that the gradient is simply the feature vector itself, which makes linear function approximation computationally efficient and conceptually simple. It also highlights the importance of designing good features, as they directly determine how the weights are updated during learning.

For Gradient Monte Carlo methods with linear function approximation, the learning process is guaranteed to converge to the global optimum of the value function approximation if
1. If the features are well-designed
2. The step size $\alpha$ is appropriately chosen

For semi gradient TD with linear function approximation, it also converges, but not the global optimum, but rather a point near the local optimum. This point is called the TD fixed point. Although the TD Fixed Point is not the global minimum, it has been proven that this point within a bounded expansion of the lowest possible error, where

$$\bar{VE}(w_{TD}) \leq \frac{1}{1-\gamma}$$

Therefore, the asymptotic error of the TD method is no more than $\frac{1}{1 - \gamma}$ times the smallest possible error, that attained in the limit by the Monte Carlo method. However in real applications, because $\gamma$ is often near one, this expansion factor can be quite large, so there is substantial potential loss in asymptotic performance with the TD method. So, we should carefully select which method to use depend on the problem itself and the desired convergence


For semi-gradient TD methods using linear function approximation, convergence is still guaranteed under certain conditions, but not to the global minimum of the Mean Squared Value Error (MSVE). Instead, the algorithm converges to a point near the global optimum known as the TD fixed point.

The TD fixed point represents a stable set of weights $w_{TD}$ where updates no longer change significantly. While this point may not perfectly minimize the error, it is provably close to the true minimum, with the performance bound

$$\bar{VE}(w_{TD}) \leq \frac{1}{1 - \gamma} \, \bar{VE}(w_{MC})$$

* $\bar{VE}(w_{TD})$: the MSVE at the TD fixed point
* $\bar{VE}(w_{MC})$: the achievable global minimum by the linear Monte Carlo method

This inequality guarantees that the asymptotic error of TD will be no more than $\frac{1}{1 - \gamma}$ times the minimal error. In practice, when $\gamma$ is small (far from 1), the bound is tight, and TD performs close to optimal; when $\gamma$ is near 1 (long-term planning problems), the bound becomes large, meaning TD could potentially be much worse than the Monte Carlo solution. Thus, while TD methods are often preferred due to their online learning capability and lower variance, there is a trade-off in asymptotic performance. In problems with very high discount factors, it is important to carefully consider whether Monte Carlo or TD is more appropriate for the desired level of convergence.

## Feature Construction for Function Approximation
We use features to represent a state. The way we represent states through features is crucial for reinforcement learning, as it directly impacts training speed, stability, and the ability to incorporate prior domain knowledge. Good feature construction can enable an RL agent to generalize well, especially when dealing with large or continuous state spaces.

In linear function approximation, each feature is treated independently, meaning the model cannot naturally capture interactions between features. To address this, we often engineer or combine features to create richer representations that capture these relationships.


### State Aggregation
State aggregation is one of the simplest techniques for feature construction. It works by grouping similar states together and treating them as if they were the same single state. This drastically reduces the number of distinct states, which makes learning more computationally efficient and improves generalization in large or continuous state spaces.

In state aggregation, the core idea is to partition the state space into non-overlapping groups or aggregated states. This means each original state is assigned to exactly one group, and no state can belong to multiple groups at the same time.

<img src="https://raw.githubusercontent.com/wayexists02/my-study-note/image/typora/image/image-20200303011108051.png" width=500>

However, aggregation must be done carefully because too coarse aggregation makes the agent overgeneralizes by treating very different states as identical, which can lead to poor performance. Too fine aggregation, on the other hand, makes the agent essentially treats every state as unique, losing the benefit of generalization and requiring much more data. The key is to find a balance between generalization and discrimination.

### Coarse Coding
Coarse coding is a way to represent states using overlapping features, which allows for more flexible and powerful generalization compared to non-overlapping methods like state aggregation. In coarse coding, each feature represents a region of the state space, and multiple features can cover the same state. This means a single state may activate several features at once. When an update occurs, all active features are adjusted, which in turn affects all states that share those features. As a result, learning about one state immediately improves the value estimates of nearby states. This overlapping structure provides smoother generalization, where similar states are treated similarly.

The shape and size of these overlapping regions determine how the agent generalizes across states and how well it can discriminate between them:
* Small, distinct regions: Provide fine-grained discrimination, allowing the agent to distinguish small differences between states. Suitable for tasks requiring precise control or where small differences are critical.
* Large, broad regions: Lead to stronger generalization, as many states share the same features. Useful when the environment is noisy or when learning must be fast with limited data.

By tuning the size and shape of these overlapping features, we can control the trade-off between generalization and discrimination

<img src="https://raw.githubusercontent.com/wayexists02/my-study-note/image/typora/image/image-20200303011256015.png" width=500>

### Tile Coding
Tile coding is a type of coarse coding technique that creates overlapping state representations to improve generalization. It works by applying state aggregation (tiling) multiple times, with each tiling slightly offset from the others. In each tiling, the state space is divided into non-overlapping tiles (like a grid). Therefore, a given state activates exactly one tile in each tiling, meaning multiple features are active for the same state across different tilings. When the agent updates its value estimate, all active tiles are updated, which simultaneously updates the estimates of nearby states that share those tiles. This structure allows the agent to generalize efficiently as updaing to one state propagate to neighboring states through overlapping tiles.

<img src="https://miro.medium.com/v2/resize:fit:1226/1*itlhmnwLnEtUaalSsivUVA.png" width=500>

The size and direction of the offset between tilings directly affect the degree and direction of generalization. Small offsets provide finer discrimination between nearby states and allow the agent to distinguish subtle differences. While larger offsets promote stronger generalization by grouping states more broadly, which is useful in noisy or high-dimensional environments. Also, the directions of offsets between each tiling directly impact the generalization directions.

Tile coding is especially practical for modern reinforcement learning systems because:
1. It is computationally efficient as only a fixed, small number of features are active per state.
2. It naturally supports incremental updates, making it ideal for online learning.
3. Its simplicity makes it easier to debug and visualize compared to complex non-linear methods like neural networks.

### Neural Networks for Feature Representation
Another way to represent features in reinforcement learning is by using a neural network. In this approach, we can directly feed raw state data into the network rather than relying on hand-crafted features. The network automatically learns feature representations implicitly through the process of backpropagation, discovering patterns and relationships within the data as it trains.

The learning process is framed as a supervised learning problem, where:
* Input: The raw state or its basic attributes.
* Output: The predicted value for that state, such as $\hat{v}(s; w)$.
* Objective: Minimize the value error, so the network’s predictions get closer to the true expected return.


Compared to coarse coding or other manual feature engineering methods, the neural networks have the advantages of
1. Capture complex, non-linear relationships that simpler methods cannot represent.
2. Eliminate the need for manual feature construction, as the network itself discovers useful features during training.
3. Suitable for unstructured and high-dimensional inputs, such as images or raw sensor data.

But neural network also has the disadvantages of
1. Prone to unstable learning, especially in RL where training data is non-stationary and bootstrapped targets are used.
2. The learned features are often uninterpretable, making it difficult to understand why certain decisions are made.
3. Require careful tuning of hyperparameters and large amounts of data for effective learning.

## On-Policy Control with Approximation
On-policy prediction methods, such as gradient Monte Carlo and semi-gradient TD, allow us to estimate the state-value function $v_\pi(s)$ for a fixed policy $\pi$. These methods are useful for evaluating how good a given policy is, but by themselves, they do not improve the policy because knowing the state-value only tells the agent which state is good, but doesn't tell it how to get there.

To enable control, the agent must not only evaluate a policy but also improve it iteratively. This requires estimating action-values $q_\pi(s, a)$ instead of just state values. The action-value function represents the expected return for taking a specific action a in a given state s and then following the policy thereafter.

By approximating $q_\pi(s, a)$ using function approximation techniques, the agent can generalize across large or continuous state-action spaces. This makes it possible to handle environments where storing exact values for every state-action pair would be infeasible.

### Feature Representation for State-Action Pairs
To approximate the action-value function using function approximation, we extend the idea of state-value approximation. Just like with state values, we estimate the action value as the dot product between the weights and a feature vector

$$\hat{q}(s, a; w) = w^\top x(s, a)$$

Here, $x(s, a)$ is the feature representation of the state-action pair.

Given a feature vector $x(s)$ that represents a state $s$ using $n$ features, we also need to inform the function approximator which action was taken. To do this, we can stack multiple copies of the state feature vector, one for each possible action. Suppose there are $m$ possible actions for each state, then the stacked feature vector, $x(s, a)$, will then have size $(n \times m, 1)$, where

$$
x(s, a) =
\begin{bmatrix}
0 \\ \vdots \\ 0 \\ x(s) \\ 0 \\ \vdots \\ 0
\end{bmatrix} \in \mathbb{R}^{n \cdot m}
$$

Note that for a given state-action pair $(s, a)$, only the block corresponding to action $a$ will contain the actual state features $x(s)$, and all other blocks will be filled with zeros. This creates a sparse vector that clearly separates different actions, effectively giving each action its own set of weights.

Stacking has the advantages of
1. Clear, interpretable separation of actions.
2. Effective for linear approximation and structured feature methods like tile coding.

But also has the issues of
1. Feature vector size grows linearly with the number of actions $n \cdot m$.
2. Inefficient for environments with very large action spaces, leading to many zeros and wasted computation.

Thus, the approximator learns separate estimators for each state-action pair, just like in tabular Q-learning but with generalization across states.

As an alternative for stacking, we can encode the action directly and append it to the original state feature vector $x(s)$. For example, we can use a one-hot vector or a learned action embedding to represent the action and concatenate it with the state features to form a compact representation.

This works especially well with neural networks, which can handle non-linear relationships and automatically learn how state and action features interact. However, for linear function approximators, this approach is much weaker as a linear function cannot easily capture the complex patterns between state and action from a single concatenated vector. In such cases, stacking remains the more effective choice.

### Episodic Semi-Gradient SARSA

Episodic Semi-Gradient SARSA is a control algorithm that uses function approximation to estimate the action-value function $\hat{q}(s, a; w)$.  
It extends the tabular SARSA method by replacing the table of Q-values with a parameterized function whose weights $w$ are updated using gradient-based methods.

The weight update rule is similar to the semi-gradient TD update, but instead of using the estimated state value $v(s)$, we use the esimated action-value $\hat{q}(s, a; w)$, where

$$w \leftarrow w + \alpha \big[ R_{t+1} + \gamma \hat{q}(S_{t+1}, A_{t+1}; w) - \hat{q}(S_t, A_t; w) \big] \nabla_w \hat{q}(S_t, A_t; w)$$

For a fixed policy, this method converges in the same way as TD(0).


### Episodic Semi-Gradient SARSA Algorithm

1. Given:
   - Initial policy $\pi$ (e.g., $\epsilon$-greedy)
   - Learning rate $\alpha \in (0, 1]$
   - Discount factor $\gamma \in [0, 1]$
   - A parameterized action-value function $\hat{q}(s, a; w)$

2. Initialize:
   - Weights $w$ arbitrarily for all $s \in \mathcal{S}, a \in \mathcal{A}$.

3. Repeat (for each episode):
   - Initialize the starting state $S_0$
   - Choose initial action $A_0 \sim \pi(\cdot \mid S_0)$

   For each timestep $t$:
   - Take action $A_t$, observe reward $R_{t+1}$ and next state $S_{t+1}$.
   - Choose next action $A_{t+1} \sim \pi(\cdot \mid S_{t+1})$ based on the current policy.
   - Update the weights:
     $$w \leftarrow w + \alpha \big[ R_{t+1} + \gamma \hat{q}(S_{t+1}, A_{t+1}; w) - \hat{q}(S_t, A_t; w) \big] \nabla_w \hat{q}(S_t, A_t; w)$$
   - Update policy $\pi$ to be greedy or $\epsilon$-greedy with respect to the current $\hat{q}(s, a; w)$.

4. Repeat until convergence:
   - Continue running episodes until $\hat{q}(s, a; w)$ stabilizes.

### Extending to Q-Learning and Expected SARSA
The same semi-gradient approach can be applied to other control algorithms like Q-learning and Expected SARSA by simply changing the target value used in the update rule. The structure of the weight update remains the same, but the way we estimate the next state's action value differs.

#### 1. Semi-Gradient Q-Learning
Q-learning is off-policy, meaning it updates the value of the current state-action pair using the best possible action at the next state, independent of the policy used to select actions during exploration. The update rule is

$$w \leftarrow w + \alpha \big[ R_{t+1} + \gamma \max_{a'} \hat{q}(S_{t+1}, a'; w) - \hat{q}(S_t, A_t; w) \big] \nabla_w \hat{q}(S_t, A_t; w)$$


#### 2. Semi-Gradient Expected SARSA
Expected SARSA is a generalization of SARSA that uses the expected action-value under the current policy at the next state instead of relying on a single sampled action. The update rule is

$$w \leftarrow w + \alpha \big[ R_{t+1} + \gamma \sum_{a'} \pi(a'|S_{t+1}) \hat{q}(S_{t+1}, a'; w) - \hat{q}(S_t, A_t; w) \big] \nabla_w \hat{q}(S_t, A_t; w)$$

## Systematic Exploration In Functional Approximation
Systematic exploration is critical in functional approximation as it forces the agent to explore and therefore obtain a better knowledge of the full environment. One way of encouraging systematic exploration is through optimisic initialization, which is done by initializing all the estimated state-values or action values to be higher than the actual ones. Similar to the tabular case, this makes the agent optimistic about all the states or state-action pairs so it will explore them in the beginning and gradually update the estimation to it's true values. However, this is not as simple as the tabular case due to two issues.
1. We can only intiailize our optimistic initial values by initializing the weights $w$ of the function approximator. However in if the function approximator is a complex, non-linear function, like a neural network. We cannot gaurenteen that we can initialize the weights so the predicted value is always optimistic for all possible inputs
2. Function approximation uses generalization for faster learning, but it also causes problems for exploration because the agent can think optimistically for one states and visit it and updates its estimation based on the observed return and transitions. However, the nearby states' values will also be updated due to the generalization, and therefore their updated values may not be optimistic any more although the state itself is not yet visited.

$\epsilon$-greedy is another way that enforces exploration since the agent is foreced to taken a random choice of action from time to time. $\epsilon$-greedy works for all cases of function approximation because it only depends on the final estimated values from the function approximator and therefore not impacted by generalization. However, the issue with $\epsilon$-greedy is that its exploration purely depends on randomness, which is not systematic.

## Systematic Exploration in Function Approximation
Systematic exploration is critical in reinforcement learning with function approximation because it ensures that the agent explores different parts of the environment instead of getting stuck in a small subset of states or actions. This broader exploration helps the agent build a more accurate understanding of the environment and improves the quality of the learned policy.

### Optimistic Initialization
One common way to encourage exploration is through optimistic initialization, where the initial value estimates for all states or state-action pairs are set higher than their true values. This makes the agent initially optimistic about every unexplored state or action, motivating it to try them out in the beginning. Over time, as the agent gathers real experience, these optimistic estimates are gradually corrected toward their true values.

While optimistic initialization works well in tabular methods, it becomes more challenging with function approximation due to two main issues:

1. Weight Initialization Limitation: in function approximation, we don't directly store a separate value for each state or state-action pair. Instead, we initialize the weights $w$ of the function approximator. For simple linear models, we may be able to set weights so that initial predictions are optimistic. However, for complex, non-linear models like neural networks, there is no guarantee that initializing the weights will produce optimistic predictions across all possible states. This makes it hard to ensure consistent optimistic initialization.

2. Impact of Generalization: function approximation relies on generalization, so updating the estimate for one state can also affect the values of nearby or similar states. For example, when the agent visits a state and updates its value based on the observed return, neighboring states may also have their values updated indirectly. This can cause the optimistic initial values of unvisited states to disappear prematurely, even though those states have never been explored. As a result, the agent might stop exploring too soon.

### $\epsilon$-Greedy Exploration
An alternative approach is $\epsilon$-greedy exploration since it does not depend on optimistic initialization or how the weights are set. It only relies on the final outputs of the function approximator, making it robust to generalization issues. However, the $\epsilon$-greedy explorations is random, not systematic.

## Average Reward
Average reward is another way of formalizing a continuing task without using discounting, which stabilizes the return and focuses on long-term steady-state performance. Instead of reducing the value of future rewards with a discount factor $\gamma$, we evaluate a policy based on the average reward per time step that the agent receives when following that policy indefinitely. The average reward of a policy $\pi$ is defined as

$$r(\pi) = \sum_{s \in \mathcal{S}} \mu_{\pi}(s) \sum_{a \in \mathcal{A}} \pi(a|s) \, \mathbb{E}[R_{t+1} \mid S_t = s, A_t = a]$$

$\mu_{\pi}(s)$ is the stationary distribution of states under policy $\pi$, representing how frequently each state is visited in the long run. By weighting rewards using $\mu_{\pi}(s)$, states that are visited more often have greater influence on the average reward.

The average reward of a policy, $r(\pi)$ is a single scalar value. So, we can easily compares two policies by comparing their average reward, and the policy with the higher average reward is better. The optimal policy, $\pi_*$, is then the one that maximizes the average reward

$$\pi_* = \arg\max_\pi r(\pi)$$

Since there is no discounting, we redefine returns using the differential return, which measures the cumulative difference between observed rewards and the average reward

$$G_t = \sum_{k=0}^{\infty} \big( R_{t+k+1} - r(\pi) \big)$$

This formulation focuses on relative improvements compared to the average reward rather than just raw accumulated reward.

Using the differential return, we can write the Bellman equations for the average reward setting. These equations are very similar to the discounted case but without the discount factor, and using the average reward term instead, where

$$v_{\pi}(s) = \sum_{a} \pi(a|s) \sum_{s', r} p(s', r \mid s, a) \big[ r - r(\pi) + v_{\pi}(s') \big]$$

$$q_{\pi}(s, a) = \sum_{s', r} p(s', r \mid s, a) \big[ r - r(\pi) + \sum_{a'} \pi(a'|s') q_{\pi}(s', a') \big]$$

$$v_*(s) = \max_a \sum_{s', r} p(s', r \mid s, a) \big[ r - r(\pi_*) + v_*(s') \big]$$

$$q_*(s, a) = \sum_{s', r} p(s', r \mid s, a) \big[ r - r(\pi_*) + \max_{a'} q_*(s', a') \big]$$

### Using Average Reward for Control
The updates using average reward are very similar to those in the discounted case, with a key difference on how the update is computed. In standard Q-learning with discounting, the update rule is

$$
q(S_t, A_t) \leftarrow q(S_t, A_t) + \alpha \big[ R_{t+1} + \gamma \max_{a’} q(S_{t+1}, a’) - q(S_t, A_t) \big]
$$

Here, the discount factor $\gamma$ down-weights future rewards, emphasizing immediate rewards more strongly. In average reward methods, there is no discount factor $\gamma$. Instead, we subtract the current estimate of the average reward $r(\pi)$ of the policy at each step, where

$$
q(S_t, A_t) \leftarrow q(S_t, A_t) + \alpha \big[ R_{t+1} - r(\pi) + \max_{a’} q(S_{t+1}, a’) - q(S_t, A_t) \big]
$$

By subtracting the average reward, the updates center the rewards around zero, which keeps the learning process stable and focuses the agent on relative improvements over the long-term steady-state performance rather than absolute return.

In addition to updating the action-value function, we must also update the estimate of the average reward after each state–action–reward pair. This is because each new interaction with the environment provides new information about the overall performance of the policy. The update rule for the average reward is

$$r(\pi) \leftarrow r(\pi) + \beta \big(R_{t+1} - r(\pi)\big)$$

$\beta$ is a small step-size parameter. A smaller \beta ensures that the estimate of the average reward changes gradually, preventing instability or large fluctuations due to single, noisy samples.


# Policy Gradient Method
So far, all the methods we have introduced are value-based learning methods. In these methods, the agent first learns value functions, such as the state-value function $v(s)$ or the action-value function $q(s, a)$ from experience. Then, the agent selects actions based on these value estimates, and the policy is indirectly derived from the learned value functions. In other words, the value functions act as a bridge between the agent’s current state and its decision-making policy.

Instead of learning value functions and deriving a policy from them, we can directly learn the policy as a parameterized function and optimize it. This method is called the policy gradient method. We represent the policy by $\pi(a \mid s, \theta)$, which gives the probability of selecting action a given state $s$ and parameters $\theta$. Here, $\theta$ represents the learnable parameters of the policy.

For $\pi(a \mid s, \theta)$ to be a valid probability distribution, it must satisfy two constraints:

1. Non-negativity: $\pi(a \mid s, \theta) \geq 0 \quad \forall a \in \mathcal{A}, , s \in \mathcal{S}$

2. Normalization: $\sum_a \pi(a \mid s, \theta) = 1 \quad \forall s \in \mathcal{S}$

These ensure that for every state, the probability of each action is non-negative and the total probability over all actions equals 1.

To satisfy these constraints, we first define a real-valued function called the action preference, denoted as $h(s, a, \theta)$. The action preference represents how desirable it is to take action a in state s under parameters $\theta$. Since $h(s, a, \theta)$ can take any real value, we convert it into a valid probability distribution using the softmax function, where

$$\pi(a \mid s, \theta) = \frac{e^{h(s, a, \theta)}}{\sum_{a’} e^{h(s, a’, \theta)}}$$

After the softmax function, the probability distribution for all actiosn under every states is now valid.

## Advanages of Policy Gradient Method
The optimal policy for a given problem can be deterministic or stochastic. In value-based methods, improving the policy usually involves making it greedy or $\epsilon$-greedy with respect to the current action-value estimates.

However, this approach has two major limitations:
1. Difficulty in Learning Stochastic Policies: in many problems, the optimal policy is stochastic, meaning the agent must randomize its actions to perform well (e.g., Rock-Paper-Scissors, exploration in partially observable environments). Value-based methods struggle here because they force a near-deterministic improvement step by greedily selecting the best action or occasionally exploring using $\epsilon$-greedy. This makes it difficult for the agent to naturally converge to a well-calibrated stochastic policy, as the randomness is artificially imposed instead of being learned.

2. Limitations of $\epsilon$-Greedy Exploration: $\epsilon$-greedy is used in both tabular and function approximation settings to ensure exploration. However, because exploration is purely random, the agent always explores, even when it has already learned the optimal behavior. This creates a performance ceiling, as the agent never fully transitions to optimal exploitation and continues to take suboptimal random actions occasionally.


Policy gradient methods directly parameterize and optimize the policy. This allows them to adapt naturally to the type of policy required. If the optimal policy is deterministic, the learned policy will gradually converge to a pure deterministic form without requiring artificial exploration like $\epsilon$-greedy. If the optimal policy is stochastic, the method will self-adjust and discover the correct distribution of actions through experience. This flexibility makes policy gradient methods more expressive and often more suitable for complex or uncertain environments.

Also, in some problems, the value function can be extremely complex, while the policy itself is simple. In such cases, directly learning the policy is more efficient and practical than learning a full value function and deriving the policy indirectly. However, this is problem-dependent as there are also situations where value-based methods are simpler and more stable.

## Policy Gradient Objective
The goal of reinforcement learning is to find a policy that maximizes the agent’s cumulative rewards. For policy gradient methods, we directly optimize a parameterized policy $\pi(a \mid s, \theta)$. The objective function for this approach can be defined as the average reward under a given policy, where

$$r(\pi) = \sum_{s \in \mathcal{S}} \mu_{\pi}(s) \sum_{a \in \mathcal{A}} \pi(a \mid s) \sum_{s’, r} p(s’, r \mid s, a) , r$$

* $\sum_{s’, r} p(s’, r \mid s, a) r$: the expected immediate reward for taking action a in state s.
* $\pi(a \mid s) \sum_{s’, r} p(s’, r \mid s, a) r$: the expected reward at state s, weighted by the probability of selecting action a under policy $\pi$.
* $\mu_{\pi}(s)$: The stationary distribution over states under policy $\pi$, representing how frequently each state is visited when the agent follows $\pi$.


This objective function captures the long-term performance of a policy, where it first accounts for how often each state is visited $\mu_{\pi}(s)$. Then, for each state, it considers the probability of taking each action and the expected reward for that action.

By maximizing $r(\pi)$ with respect to the policy parameters $\theta$, the agent gradually adjusts its behavior to favor actions and states that yield higher rewards.

Formally, the optimization problem is:

$$\theta^* = \arg \max_\theta r(\pi_\theta)$$

## Policy Gradient Theorem
Once we define the learning objective, our next step is to optimize it by adjusting the policy parameters $\theta$. Since our goal is to maximize the objective, we use gradient ascent to update the parameters in the direction of increasing performance. The gradient of the average reward objective is

$$
\nabla_{\theta} r(\pi) = \nabla_{\theta} \sum_{s \in \mathcal{S}} \mu_{\pi}(s) \sum_{a \in \mathcal{A}} \pi(a \mid s) \sum_{s’, r} p(s’, r \mid s, a) , r
$$

Once we know the gradient of the objective, we can update the policy's parameter, $\theta$, with gradient ascent with step size of $\alpha$, where

$$\theta_{t+1} = \theta_t + \alpha \nabla_{\theta} r(\pi)$$

However, in the gradient of the average reward, the term $\mu_{\pi}(s)$ represents the state distribution under policy $\pi$ because his distribution changes whenever the policy changes, making its gradient extremely difficult to compute accurately, which is impractical.

The policy gradient theorem resolves this issue by providing a clean, analytic expression for the gradient that avoids differentiating the state distribution. It shows that

$$\nabla_{\theta} r(\pi) = \sum_{s \in \mathcal{S}} \mu_{\pi}(s) \sum_{a \in \mathcal{A}} \nabla_{\theta} \pi(a \mid s) , q_{\pi}(s, a)$$

* $q_{\pi}(s, a)$: the action-value function, representing the expected return starting from state $s$, taking action $a$, and then following policy $\pi$.
* $\nabla_{\theta} \pi(a \mid s)$: the gradient of the policy with respect to its parameters.

This form bypasses the need to compute $\nabla_{\theta} \mu_{\pi}(s)$ and gives a practical way to estimate the gradient from experience.

In the policy gradient theorem expression, $\nabla_{\theta} \pi(a \mid s)$ describes how the probability of selecting each action changes as the policy parameters $\theta$ are adjusted. 
* If a small increase in a parameter raises the probability of selecting an action, the gradient is positive for that action.
* If a small increase lowers the probability, the gradient is negative.

The action-value $q_{\pi}(s, a)$ serves as a signal that guides the gradient update, indicating whether the probability of selecting an action should be increased or decreased to improve the policy.
* If an action's expected reward is better than average, $q_{\pi}(s, a)$ will be high. This increases the probability of selecting that action in the future.
* If an action's expected reward is worse than average, $q_{\pi}(s, a)$ will be low or negative. This decreases the probability of selecting that action.

Thus, with the two term together, the policy gradient theorem naturally pushes the policy toward actions that yield higher rewards, improving the agent’s behavior over time.

The update rule is given by

$$\theta_{t+1} = \theta_t + \alpha \sum_{s \in \mathcal{S}} \mu_{\pi}(s) \sum_{a \in \mathcal{A}} \nabla_{\theta} \pi(a \mid s) , q_{\pi}(s, a)$$

## Stochastic Policy Gradient
In the current policy gradient expression, it requires summing over all possible state action pairs to computes the full gradient, which is inpractical since the state action space is often very large.
$$\nabla_{\theta} r(\pi) = \sum_{s \in \mathcal{S}} \mu_{\pi}(s) \sum_{a \in \mathcal{A}} \nabla_{\theta} \pi(a \mid s) , q_{\pi}(s, a)$$

Therefore, we can simplify the gradient computation by computing a stochastic gradient of only based on only one samples. This can be done by replacing $s$ and $a$ with an observed state action pair $(S_t, A_t)$.

$$\nabla_{\theta} r(\pi) \approx \frac{\nabla_{\theta} \pi(A_t \mid S_t)}{\pi(A_t \mid S_t)} q_{\pi}(S_t, A_t) = \nabla_{\theta} \ln{\pi(A_t \mid S_t)}q_{\pi}(S_t, A_t)$$

This formula approximates the true grtadient using only one sample, and the update rule becomes

$$\theta_{t+1} = ...$$


## Stochastic Policy Gradient

In the current policy gradient expression, the exact policy gradient sums over all state–action pairs, which is intractable in large spaces:

$$\nabla_\theta r(\pi_\theta)
= \sum_{s\in\mathcal S}\mu_{\pi_\theta}(s)\sum_{a\in\mathcal A}\pi_\theta(a\mid s),\nabla_\theta \ln \pi_\theta(a\mid s) q_{\pi_\theta}(s,a)$$

Instead, we use a stochastic estimation by sampling a single state–action pair $(S_t,A_t)$ from the on-policy distribution and replacing the expectation with that sample

$$
\nabla_\theta r(\pi_\theta)
\approx \nabla_\theta \ln \pi_\theta(A_t\mid S_t) q(S_t,A_t)
$$

where $q(S_t,A_t)$ is an observed sample at a timesteps. Now, the agent is able to update the policy online after each timestep with the newly observed state-action pair. Therefore, the update rule becomes

$$
\theta_{t+1}
= \theta_t + \alpha, \nabla_\theta \ln \pi_\theta(A_t\mid S_t) q(S_t,A_t)
$$

## Actor-Critic Algorithm

# Deep Reinforcement Learning
<img src="https://miro.medium.com/v2/resize:fit:2000/format:webp/0*P8RnG_xRY8sThfyp.png" width=500>